In [1]:
import pandas as pd
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
# pd.set_option('display.max_columns', None) 

pd.options.display.max_columns = None

from IPython.core.display import HTML

from django.contrib.auth.hashers import make_password


In [2]:
import datetime
from pytz import timezone
from decimal import *   

def get_active_season_id():
    season_obj = Season.objects.get(is_active=True)
    return season_obj.id

In [23]:
def get_total(date):
    total_value = ProcurementGroup.objects.filter(procurement_date=date).aggregate(Sum('produce_net_weight'))[
        'produce_net_weight__sum']
    return total_value

In [8]:
def get_harvest_list(ticket_number, date):
    data = []
    if Harvest.objects.filter(ticket_number=ticket_number).exists():
        harvest_values = list(
            Harvest.objects.filter(ticket_number=ticket_number).order_by('date_of_harvest').values_list('id',
                'sowing__farmer__first_name', 'sowing__farmer__code','date_of_harvest', 'value', 'unit__name', 'ticket_number'))
        harvest_columns = ['harvest_id', 'farmer_name', 'farmer_code','date_of_harvest', 'value', 'unit_name', 'ticket_number']
        harvest_df = pd.DataFrame(harvest_values, columns=harvest_columns)
        data = harvest_df.to_dict('r')
    return data

In [9]:
def get_harvest_status(ticket_number, str):
    data = False
    if Harvest.objects.filter(ticket_number=ticket_number).exists():
        harvest_total = Harvest.objects.filter(ticket_number=ticket_number).aggregate(Sum('value'))['value__sum']
        if harvest_total == str:
            data = True
    return data

### procurement_log

In [10]:
data_dict_from_interface = {
    'procurement_date': "2020-09-01",
    'str_weight': '',
    'agent_id': '',
    'ticket_number': '',
    'vehicle_number': '',
    'net_weight': ''
}

Procurement_obj = Procurement.objects.filter()

for data in data_dict_from_interface:
    if data == 'procurement_date':
        Procurement_obj = Procurement_obj.filter(procurement_group__procurement_date=data_dict_from_interface[data])
    elif data == 'str_weight':
        Procurement_obj = Procurement_obj.filter(str_weight=data_dict_from_interface[data])
    elif data == 'agent_id':
        Procurement_obj = Procurement_obj.filter(procurement_group__agent_id=data_dict_from_interface[data])
    elif data == 'ticket_number':
        Procurement_obj = Procurement_obj.filter(ticket_number=data_dict_from_interface[data])
    elif data == 'vehicle_number':
        Procurement_obj = Procurement_obj.filter(vehicle_number=data_dict_from_interface[data])
    elif data == 'net_weight':
        Procurement_obj = Procurement_obj.filter(procurement_group__produce_net_weight=data_dict_from_interface[data])

procurement_values = list(
    Procurement_obj.values_list(
        'id', 'ticket_number', 'vehicle_number', 'vehicle_driver_name', 'str_weight', 'str_weight_unit__name',
        'remark',
        'empty_vehicle_timestamp', 'empty_vehicle_weight_data_inputed', 'empty_vehicle_weight',
        'loaded_vehicle_timestamp',
        'loaded_vehicle_weight_data_inputed', 'loaded_vehicle_weight', 'gunnybag_count', 'gunnybag_weight',
        'moisture',
        'produce_gross_weight', 'unit__name', 'procurement_group_id', 'procurement_group__agent_id',
        'procurement_group__agent__first_name',
        'procurement_group__procurement_date', 'procurement_group__procurement_produce__name',
        'procurement_group__procurement_produce__id', 'procurement_group__season_id',
        'procurement_group__produce_net_weight',
        'procurement_group__price_per_unit', 'procurement_group__unit_for_pricing',
        'procurement_group__cost', 'procurement_group__payment_to_wallet', 'procurement_group__payment_to_agent',
        'procurement_group__agent_price_deduction','have_other_agent_farmer'))


procurement_columns = ['procurement_id', 'ticket_number', 'vehicle_number', 'vehicle_driver_name', 'str_weight',
                       'str_weight_unit_name', 'remark',
                       'empty_vehicle_timestamp', 'empty_vehicle_weight_data_inputed', 'empty_vehicle_weight',
                       'loaded_vehicle_timestamp',
                       'loaded_vehicle_weight_data_inputed', 'loaded_vehicle_weight', 'gunnybag_count',
                       'gunnybag_weight', 'moisture',
                       'produce_gross_weight', 'unit_name', 'procurement_group_id', 'agent_id', 'agent_username',
                       'procurement_date', 'procurement_produce_name',
                       'procurement_produce_id', 'season_id', 'produce_net_weight', 'price_per_unit',
                       'unit_for_pricing',
                       'cost', 'payment_to_wallet', 'payment_to_agent', 'agent_price_deduction', 'have_other_agent_farmer']

procurement_df = pd.DataFrame(procurement_values, columns=procurement_columns)
procurement_df["ticket_number"] = procurement_df["ticket_number"].astype("|S")
procurement_df['total_day_net_value'] = procurement_df.apply(lambda x: get_total(x['procurement_date']), axis=1)
procurement_df['harvest_list'] = procurement_df.apply(lambda x: get_harvest_list(x['ticket_number'], x['procurement_date']), axis=1)
procurement_df['harvest_completed'] = procurement_df.apply(lambda x: get_harvest_status(x['ticket_number'], x['str_weight']), axis=1)
procurement_df.sort_values(by=['procurement_date'], inplace=True)

data = {}
data['data'] = procurement_df.to_dict('r')
data['total_weight_for_selected_date'] = Procurement.objects.filter(procurement_group__procurement_date="2020-09-01").aggregate(
                Sum('procurement_group__produce_net_weight'))['procurement_group__produce_net_weight__sum']
# procurement_df

### Farmer consolidated report

In [15]:
    #famer_data
    harvest_table_list = list(Harvest.objects.filter().values_list('sowing__farmer__code', 'sowing__farmer__first_name', 'sowing__farmer__last_name', 'sowing__farmer__village__name', 'sowing__season__name', 'value', 'date_of_harvest', 'sowing__area', 'sowing__farmer__cluster__name'))
    column_list = ['farmer_code', 'farmer_first_name', 'farmer_last_name', 'farmer_village', 'season', 'yeald', 'date_of_harvest', 'area', 'cluster_name']
    harvest_data_frame = pd.DataFrame(harvest_table_list, columns=column_list)

    #agent_data
    agent_farmer_list = list(AgentFarmerMap.objects.filter(farmer__farmer__code__in=harvest_data_frame['farmer_code']).values_list('farmer__farmer__code', 'agent__first_name'))
    agent_farmer_columns = ['farmer_code', 'agent_name']
    agent_farmer_df = pd.DataFrame(agent_farmer_list, columns=agent_farmer_columns)

    #supervisor_data
    supervisor_farmer_list = list(UserFarmerMap.objects.filter(farmer__farmer__code__in=harvest_data_frame['farmer_code']).values_list('farmer__farmer__code', 'officer__username'))
    supervisor_farmer_colums = ['farmer_code', 'supervisor_name'] 
    supervisor_farmer_df = pd.DataFrame(supervisor_farmer_list, columns=supervisor_farmer_colums)

    #merged_data
    farmer_agent_map_df = pd.merge(harvest_data_frame, agent_farmer_df, left_on='farmer_code', right_on='farmer_code')
    supervisor_fatmer_agent_map_df = pd.merge(farmer_agent_map_df, supervisor_farmer_df, left_on='farmer_code', right_on='farmer_code')

    #sum yeald and find last harvest date finally merge with supervisor_fatmer_agent_map_df
    grouped_df = supervisor_fatmer_agent_map_df.groupby(['farmer_code']).agg({'yeald': 'sum', 'date_of_harvest': 'max'}).rename(columns={'yeald': 'total_yeald', 'date_of_harvest': 'last_harvest_date'})
    final_supervisor_fatmer_agent_map_df = pd.merge(supervisor_fatmer_agent_map_df, grouped_df, left_on='farmer_code', right_index=True)
    final_supervisor_fatmer_agent_map_df = final_supervisor_fatmer_agent_map_df.drop(columns=['yeald', 'date_of_harvest'])
    final_supervisor_fatmer_agent_map_df = final_supervisor_fatmer_agent_map_df.drop_duplicates()

#     #final_df
#     final_supervisor_fatmer_agent_map_df = final_supervisor_fatmer_agent_map_df.reindex(columns=['farmer_code', 'farmer_first_name', 'farmer_last_name', 'farmer_village', 'cluster_name', 'agent_name', 'supervisor_name', 'last_harvest_date', 'area', 'total_yeald'])
    
    #final_df
    df = final_supervisor_fatmer_agent_map_df.reindex(columns=['farmer_code', 'farmer_first_name', 'farmer_last_name', 'farmer_village', 'cluster_name', 'agent_name', 'supervisor_name', 'last_harvest_date', 'area', 'total_yeild'])
    
    writer = pd.ExcelWriter(str('static/media/') + 'farmer_wise_consolidated_yeild.xlsx', engine='xlsxwriter')

    # creating excel sheet with name
    df.to_excel(writer, index=False, sheet_name='Sheet1', startrow=1)

    # assigning that sheet to obj
    workbook = writer.book
    worksheet = writer.sheets['Sheet1']
    merge_format = workbook.add_format({
        'bold': 1,
        'border': 1,
        'align': 'center',
        'valign': 'vcenter',
        'fg_color': 'yellow'})

    # Merge 3 cells.
    worksheet.merge_range('A1:K1', 'Farmer Wise Consolidated Yeild Report', merge_format)
    format1 = workbook.add_format({'num_format': '#,##0.00'})

    # Set the column width and format.
    worksheet.set_column('B:B', 18, format1)
    worksheet.set_column(0, 27, 20)

    # Add a header format.
    header_format = workbook.add_format({
        'fg_color': '#D7E4BC'})

    # Write the column headers with the defined format.
    for col_num, value in enumerate(df.columns.values):
        worksheet.write(0, col_num + 1, value, header_format)
    writer.save()

    df

,farmer_code,farmer_first_name,farmer_last_name,farmer_village,cluster_name,agent_name,supervisor_name,last_harvest_date,area,total_yeild
0,20BPR001,madeshappa,mallappa,Katnavadi,Bommanahalli,Rajashekhara Murthy,prasad,2020-08-13,1.5,NaN
12,20NMM001,Basavaraju,Chanappa,Mallahalli,Nanjangud,Manjesh M,mahadevaprasad,2020-07-21,1.0,NaN
20,20NMM002,Shivaramu,Mahadevappa,Kelupura,Nanjangud,Manjesh M,mahadevaprasad,2020-08-08,1.0,NaN
36,20BMB059,Siddaraju,puttasiddaiah,mukahalli colony,Baragi,K Basappa,madappa,2020-08-07,0.5,NaN
46,20YMG001,Nandisha,Siddappa,Channagundi Village,Yadalaya,Gopalaa,mahadevaprasad,2020-07-07,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...
21431,20BAM228,sanju elwin,famji elwin,mangla,Berambadi,Madappa,ajith,2020-09-24,2.0,NaN
21432,20BAM212,santhosh,channabasappa,elchatti,Berambadi,Madappa,ajith,2020-09-27,1.0,NaN
21433,20BMB061,narayanlal,govardhanlal,Gundlupet Town,Baragi,K Basappa,madappa,2020-05-26,0.5,NaN
21434,20BAM226,nagaraju,....,lokkere,Berambadi,Madappa,ajith,2020-09-30,2.5,NaN


### Non Supplied Farmer List

In [6]:
data_dict_from_interface = {
    'cluster_id_list': [],
    'date_greater': '2020-05-1',
    'date_lesser': '2020-05-30'
}

# un-supplied farmers list
user_type_id = UserProfile.objects.get(user_id=78).user_type.id
print(user_type_id)
user_id = 78
cluster_ids = []
# if agent user type means all the agents are taken under him in userhyrarchy directly
if user_type_id == 5:
    subordinate_user_ids = [user_id]
    if UserHierarchyMap.objects.filter(superior_id__in=subordinate_user_ids).exists():
        subordinates = UserHierarchyMap.objects.filter(superior_id__in=subordinate_user_ids)
        subordinate_user_ids = list(subordinates.values_list('subordinate', flat=True))
        print(subordinate_user_ids)
else:
    if UserHierarchyMap.objects.filter(superior_id=user_id).exists():
        subordinates = UserHierarchyMap.objects.filter(superior_id=user_id)
        subordinate_user_ids = list(subordinates.values_list('subordinate', flat=True))
        if UserHierarchyMap.objects.filter(superior_id__in=subordinate_user_ids).exists():
            subordinates = UserHierarchyMap.objects.filter(superior_id__in=subordinate_user_ids)
            subordinate_user_ids = list(subordinates.values_list('subordinate', flat=True))
            if UserHierarchyMap.objects.filter(superior_id__in=subordinate_user_ids).exists():
                subordinates = UserHierarchyMap.objects.filter(superior_id__in=subordinate_user_ids)
                subordinate_user_ids = list(subordinates.values_list('subordinate', flat=True))
                if UserHierarchyMap.objects.filter(superior_id__in=subordinate_user_ids).exists():
                    subordinates = UserHierarchyMap.objects.filter(superior_id__in=subordinate_user_ids)
                    subordinate_user_ids = list(subordinates.values_list('subordinate', flat=True))

if AgentFarmerMap.objects.filter(agent_id__in=subordinate_user_ids).exists():
    print(subordinate_user_ids)
    filter_farmer_ids = list(AgentFarmerMap.objects.filter(agent_id__in=subordinate_user_ids, farmer__season_id=2).values_list('farmer__farmer_id', flat=True))

    farmer_ids = Harvest.objects.filter(sowing__season_id=2).values_list('sowing__farmer__id', flat=True)
    
    filtered_farmer_ids = list(FarmerClusterSeasonMap.objects.filter(season_id=2, farmer_id__in=filter_farmer_ids).exclude(
            farmer_id__in=farmer_ids).values_list('farmer__id', flat=True))
    
    if 'cluster_id_list' in data_dict_from_interface:
        cluster_farmer_obj =  FarmerClusterSeasonMap.objects.filter(cluster_id__in=data_dict_from_interface['cluster_id_list'])
    else:
        cluster_farmer_obj = FarmerClusterSeasonMap.objects.filter()
        
    cluster_farmer_values = list(cluster_farmer_obj.filter(farmer_id__in=filter_farmer_ids, season_id=2).exclude(
                farmer_id__in=farmer_ids).values_list('id', 'cluster__name', 'cluster__id', 'farmer_id',
                                                    'farmer__first_name', 'farmer__last_name', 'farmer__village__name',
                                                    'farmer__code'))
        
    cluster_farmer_columns = ['id', 'cluster_name', 'cluster_id', 'farmer_id', 'farmer_first_name', 'farmer_last_name',
                            'farmer_village_name', 'farmer_code']
    
    cluster_farmer_df = pd.DataFrame(cluster_farmer_values, columns=cluster_farmer_columns)
    
    sowing_obj = Sowing.objects.filter()
    
    if 'date_greater' in data_dict_from_interface:
        sowing_obj = sowing_obj.filter(sowing_date__gte=data_dict_from_interface['date_greater'])
    if 'date_lesser' in data_dict_from_interface:
        sowing_obj = sowing_obj.filter(sowing_date__lte=data_dict_from_interface['date_lesser'])

    sowing_values = list(sowing_obj.filter(farmer_id__in=filtered_farmer_ids, season_id=2, cultivation_phase_id=2).values_list('id',
                                                                                                                'farmer_id',
                                                                                                                'sowing_date',
                                                                                                                'area',
                                                                                                                'cultivation_phase_id',
                                                                                                                'cultivation_phase__name'))
    sowing_columns = ['sowing_id', 'farmer_id', 'sowing_date', 'area', 'cultivation_phase_id', 'cultivation_phase_name']
    sowing_df = pd.DataFrame(sowing_values, columns=sowing_columns)

    # converted_today = datetime.datetime.strptime(, '%Y-%m-%d')
    today = datetime.datetime.now().date()
    sowing_df['crop_age'] = today - sowing_df['sowing_date']
    sowing_df['crop_age'] = sowing_df['crop_age'].astype('timedelta64[D]')
    sowing_df['crop_age'] = sowing_df['crop_age'].astype(int)
    farmer_sowing_merged_df = pd.merge(cluster_farmer_df, sowing_df, left_on='farmer_id', right_on='farmer_id',
                                    how="left")

    farmer_cluster_map_ids = list(
        FarmerClusterSeasonMap.objects.filter(season_id=2).exclude(farmer_id__in=farmer_ids).values_list('id',
                                                                                                        flat=True))
    agent_farmer_map_values = list(
        AgentFarmerMap.objects.filter(farmer_id__in=farmer_cluster_map_ids).values_list('id', 'farmer__farmer_id',
                                                                                        'agent__first_name',
                                                                                        'agent__last_name'))
    agent_farmer_map_columns = ['agent_map_id', 'farmer_id', 'agent_first_name', 'agent_last_name']
    agent_farmer_map_df = pd.DataFrame(agent_farmer_map_values, columns=agent_farmer_map_columns)

    df = pd.merge(farmer_sowing_merged_df, agent_farmer_map_df, left_on='farmer_id', right_on='farmer_id', how="left")
    df = df.drop(['id', 'cluster_id', 'farmer_id', 'sowing_id', 'cultivation_phase_id', 'agent_map_id'], axis=1)
    df = df.fillna(0)
    writer = pd.ExcelWriter(str('static/media/') + 'Non-suplied farmers.xlsx', engine='xlsxwriter')

    # assigning that sheet to obj
    workbook = writer.book

    merge_format = workbook.add_format({
        'bold': 1,
        'border': 1,
        'align': 'center',
        'valign': 'vcenter',
        'fg_color': 'yellow'})

    header_format = workbook.add_format({
        'fg_color': '#D7E4BC'})
    format1 = workbook.add_format({'num_format': '#,##0.00'})
    if user_type_id == 5:
        subordinate_user_ids = [user_id]
        clusters = UserClusterMap.objects.filter(user_id=user_id).values_list('cluster__name', flat=True)
        df = df[df['cluster_name'].isin(clusters)]
    for cluster in df['cluster_name'].unique():
        cluster_based_df = df[df['cluster_name'] == cluster]
        cluster_based_age_based_df = cluster_based_df[cluster_based_df['crop_age'] > 35]
        cluster_based_age_based_df.to_excel(writer, sheet_name=str(cluster), startrow=1)
        cluster = writer.sheets[str(cluster)]
        cluster.merge_range('A1:L1', 'farmers', merge_format)
        format1 = workbook.add_format({'num_format': '#,##0.00'})
        cluster.set_column('B:B', 18, format1)
        cluster.set_column(0, 11, 20)

        # Write the column headers with the defined format.
        for col_num, value in enumerate(df.columns.values):
            cluster.write(0, col_num + 1, value, header_format)
    writer.save()
    document = {}
    try:
        image_path = str('static/media/') + 'Non-suplied farmers.xlsx'
        with open(image_path, 'rb') as image_file:
            encoded_image = b64encode(image_file.read())
            document['excel'] = encoded_image
    except Exception as err:
        print(err)
    document['data'] = df.to_dict('r')
df

1
[22, 23, 24, 25, 26, 27, 28, 29, 30, 40, 37, 39, 31, 57, 35, 34, 36, 38, 77, 88, 41, 32, 111, 121, 71, 72, 73, 44, 43, 48, 46, 106, 104, 110, 58, 60, 59, 56, 53, 47, 103, 120, 54, 65, 64, 90, 62, 109, 112, 122, 51, 55, 74, 49, 50, 86, 89, 107, 117, 118, 91, 113, 114, 63, 85, 124, 125, 126, 123, 87]
name 'b64encode' is not defined


,cluster_name,farmer_first_name,farmer_last_name,farmer_village_name,farmer_code,sowing_date,area,cultivation_phase_name,crop_age,agent_first_name,agent_last_name
0,HD Kote,Gopalaiha,Dasaiha,jiyaracoloni,20HSM020,2020-05-20,0.5,Transplanted,148.0,M Guru,Mahadevappa K B
1,Berambadi,seena,krishashetty,Bheemanabeedu,20BAA021,2020-05-23,0.5,Transplanted,145.0,Arasashetty,Kongalashetty
2,Bommanahalli,rajesh,gurulingappa,Sheelavanthapura,20BPR034,2020-05-25,2.0,Transplanted,143.0,Rajashekhara Murthy,Puttaswamappa
3,Nanjangud,Giri,Madappa,Aralikate,20NMM042,2020-05-05,1.5,Transplanted,163.0,M Kumaraswamy,Mudddappa
4,Yadalaya,Ganesh,Mahadevayya,Hunasuru circle,20YMG059,0,0.0,0,0.0,Gopalaa,Hanumaiah
...,...,...,...,...,...,...,...,...,...,...,...
373,Terakanambi,basavanna s,shortage seedlings,Sagade,20TNB091,2020-06-30,1.0,Transplanted,107.0,Basavanna S,Shivabasappa
374,Terakanambi,manu,shortage seedlings,ugedanahudi,20TNM102,2020-06-23,2.0,Transplanted,114.0,Manu,Devaraju
375,Nanjangud,xyz,abc,Somahalli Village,20NMM126,0,0.0,0,0.0,Madhuchandra P,Puttaswamy Shetty
376,Sargur,ravi m s,ravi m s,Magudilu,20SPR068,2020-06-20,2.9,Transplanted,117.0,Ravi M S,Shivanagappa


In [3]:
data_dict = {
    'selected_cluster_ids': [4], 
    'selected_afs_ids': [], 
    'selected_date': None, 
    'selected_data_source_ids': []
}

data = {'selected_cluster_ids': []}
    
sowing_geo_fence_map_obj = SowingBoundaryMap.objects.filter()
if data_dict['selected_date'] != None or data_dict['selected_date'] !='':
    sowing_geo_fence_map_obj = sowing_geo_fence_map_obj.filter(sowing__geo_fence_done_on=data_dict['selected_date'])
    
if data_dict['selected_afs_ids']:
    sowing_geo_fence_map_obj = sowing_geo_fence_map_obj.filter(sowing__farmer__created_by_id__in=data_dict['selected_afs_ids'])
    
if data_dict['selected_data_source_ids']:
    sowing_geo_fence_map_obj = sowing_geo_fence_map_obj.filter(data_source_id__in=data_dict['selected_data_source_ids'])
    
sowing_boundry_ids = sowing_geo_fence_map_obj.values_list('sowing_boundry', flat=True)
field_ids = sowing_geo_fence_map_obj.values_list('sowing_id', flat=True)

farmer_ids = list(set(list(sowing_geo_fence_map_obj.values_list('sowing__farmer_id', flat=True))))
        

sowing_geo_fence_map_list = list(sowing_geo_fence_map_obj.values_list('sowing__farmer_id','sowing_id', 'sowing__area_calculated_via_geo_fencing','data_source','sowing__farmer__first_name', 'sowing__farmer__last_name', 'sowing__farmer__village__name','sowing__farmer__created_by__username', 'sowing__farmer__created_by_id'))
sowing_geo_fence_map_column = ['farmer_id', 'sowing_id', 'area_in_acre','data_source', 'first_name', 'last_name', 'farmer_village','created_by_name', 'created_by_id']
sowing_geo_fence_map_df = pd.DataFrame(sowing_geo_fence_map_list, columns=sowing_geo_fence_map_column)

if data_dict['selected_cluster_ids']:
    print('cluster filter applied')
    farmer_cluster_season_map = FarmerClusterSeasonMap.objects.filter(farmer_id__in=farmer_ids, cluster_id__in=data_dict['selected_cluster_ids'])
else:
    farmer_cluster_season_map = FarmerClusterSeasonMap.objects.filter(farmer_id__in=farmer_ids)
    
cluster_farmer_list = list(farmer_cluster_season_map.values_list('farmer_id', 'cluster_id', 'cluster__name'))
cluster_farmer_columns = ['farmer_id', 'cluster_id', 'cluster_name']
cluster_df = pd.DataFrame(cluster_farmer_list, columns=cluster_farmer_columns)

farmer_cluster_season_map_df = pd.merge(sowing_geo_fence_map_df, cluster_df, how='left')

sowing_boundry_obj = SowingBoundary.objects.filter(id__in=sowing_boundry_ids).order_by('id')

sowing_boundry_list = list(sowing_boundry_obj.values_list('sowing_id', 'latitude', 'longitude'))
sowing_boundry_column = ['sowing_id', 'lat', 'lng']
sowing_boundry_df = pd.DataFrame(sowing_boundry_list, columns=sowing_boundry_column)

if not sowing_boundry_df.empty:
    sowing_boundry_series = sowing_boundry_df.groupby('sowing_id').apply(lambda x: x.to_dict('r')).to_frame()
else:
    sowing_boundry_series = sowing_boundry_df

final_df = farmer_cluster_season_map_df.merge(sowing_boundry_series, how='left', left_on='sowing_id', right_on='sowing_id')
final_df['fillColor'] = '#14b806'
final_df.loc[final_df.data_source == '1', 'fillColor'] = '#e0e03a'
final_df.rename(columns = {0:'paths'}, inplace = True)
final_df = final_df.fillna(0)

data_dict = {
    'fence_data': final_df.to_dict('r'),
    'total_area': final_df['area_in_acre'].sum(),
    'is_data_available': final_df.empty
}

# data_dict
    
#     return Response(data=data_dict, status=status.HTTP_200_OK)

cluster filter applied


/home/suneshrajan/Software/Virtualenv/ccgb/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


### Agent Report

In [9]:
data_dict = {
    'agent_ids': [],
    'form_date': '2020-08-1',
    'to_date': '2020-08-30'
}

procurement_agent_obj = Procurement.objects.filter()

# if 'agent_ids' in data_dict:
#     if not data_dict['agent_ids']:
#         procurement_agent_obj = procurement_agent_obj.filter(procurement_group__agent_id__in=data_dict['agent_ids'])
# if 'form_date' in data_dict and 'to_date' in data_dict:
#     procurement_agent_obj = procurement_agent_obj.filter(procurement_group__procurement_date__range=[data_dict['form_date'], data_dict['to_date']])
    
procurement_agent_values_list = list(procurement_agent_obj.order_by('procurement_group__agent_id','procurement_group__procurement_date').values_list('id', 'procurement_group__agent_id', 'procurement_group__agent__first_name', 'procurement_group__agent__last_name', 'procurement_group__procurement_date', 'ticket_number', 'vehicle_number',  'vehicle_driver_name', 'str_weight', 'str_weight_unit__name', 'empty_vehicle_weight', 'loaded_vehicle_weight', 'gunnybag_count', 'gunnybag_weight', 'reason_for_weight_loss', 'moisture', 'other_deduction', 'produce_gross_weight', 'unit__name', 'procurement_group__price_per_unit', 'procurement_group__cost'))               
procurement_agent_column_list = ['procurement_id', 'agent_id', 'agent_first_name', 'agent_last_name', 'procurement_group__procurement_date', 'ticket_number', 'vehicle_number',  'vehicle_driver_name', 'str_weight', 'str_weight_unit__name', 'empty_vehicle_weight', 'loaded_vehicle_weight', 'gunnybag_count', 'gunnybag_weight', 'reason_for_weight_loss', 'moisture', 'other_deduction', 'produce_gross_weight', 'unit__name', 'price_per_unit', 'cost']

procurement_agent_df = pd.DataFrame(procurement_agent_values_list, columns=procurement_agent_column_list)

agent_df = procurement_agent_df[['agent_id', 'agent_first_name']].drop_duplicates()

data_dict = {}
for index,agent in agent_df.iterrows():
    data_dict[agent.agent_id] = {
        'agent_name': agent.agent_first_name,
        'agent_data': '',
        'total_str_weight': 0,
        'total_gunnybag_weight': 0,
        'total_produce_gross_weight': 0,
        'total_cost': 0
    }
    df = procurement_agent_df[procurement_agent_df['agent_id'] == agent.agent_id]
    
    data_dict[agent.agent_id]['data'] = df.to_dict('r')
    data_dict[agent.agent_id]['total_str_weight'] = df['str_weight'].sum()
    data_dict[agent.agent_id]['total_gunnybag_weight'] = df['gunnybag_weight'].sum()
    data_dict[agent.agent_id]['total_produce_gross_weight'] = df['produce_gross_weight'].sum()
    data_dict[agent.agent_id]['total_cost'] = df['cost'].sum()
    
agent_df

,agent_id,agent_first_name
0,31,Arasashetty
44,32,Bangaranaika
136,34,H N Papanna
184,35,Madappa
308,36,Madappa G B
309,37,Mahadevaprasad B S
427,38,H B Mahadevaswamy
440,40,Shanidevara Murthy B K
542,41,Shivamahadevu
639,43,Gopalaa


In [56]:
farmer_list = Farmer.objects.filter().values('id', 'first_name', 'last_name','business__name')

former_column = ['id', 'first_name', 'last_name', 'business_name']

df = pd.DataFrame(farmer_list, columns=former_column)

new = pd.Series(farmer_list)

# final_df = pd.merge(df, new, left_on='id', right_on=new, how='left')
# farmer_list
# df
new[0]

{'id': 2643,
 'first_name': 'Arun',
 'last_name': 'Arun',
 'business__name': 'CCGB'}

In [18]:
# def serve_cluster_based_on_loged_user(request):
user_id = 78
user_type_id = UserProfile.objects.get(user_id=user_id).user_type_id
print(user_type_id)

subordinate_user_ids = []
if user_type_id == 5:
    subordinate_user_ids = [user_id]
else:
    if UserHierarchyMap.objects.filter(superior_id=user_id).exists():
        subordinates = UserHierarchyMap.objects.filter(superior_id=user_id)
        subordinate_user_ids = subordinate_user_ids + list(subordinates.values_list("subordinate", flat=True))
        print("one")
#         print(subordinate_user_ids)
        
        if UserHierarchyMap.objects.filter(superior_id__in=subordinate_user_ids).exists():
            subordinates = UserHierarchyMap.objects.filter(superior_id__in=subordinate_user_ids)
            subordinate_user_ids = subordinate_user_ids + list(subordinates.values_list("subordinate", flat=True))
            print("two")
#             print(subordinate_user_ids)
            
            if UserHierarchyMap.objects.filter(superior_id__in=subordinate_user_ids).exists():
                subordinates = UserHierarchyMap.objects.filter(superior_id__in=subordinate_user_ids)
                subordinate_user_ids = subordinate_user_ids + list(subordinates.values_list("subordinate", flat=True))
                print("three")
#                 print(subordinate_user_ids)
                
                if UserHierarchyMap.objects.filter(superior_id__in=subordinate_user_ids).exists():
                    subordinates = UserHierarchyMap.objects.filter(superior_id__in=subordinate_user_ids)
                    subordinate_user_ids = subordinate_user_ids + list(subordinates.values_list("subordinate", flat=True))
                    print("four")
#                     print(subordinate_user_ids)

print()

sub_ids = list(UserProfile.objects.filter(user_id__in=subordinate_user_ids, user_type_id=5).values_list("user_id", flat=True))

print(sub_ids)
# print(sub_ids)
cluster_ids = set(list(UserClusterMap.objects.filter(user_id__in=sub_ids).values_list("cluster_id", flat=True)))

# print(cluster_ids)
clusters = []

for cluster in Cluster.objects.filter(id__in=cluster_ids):
#     print(cluster.id)
    cluster_dict = {}
    cluster_dict["name"] = cluster.name
    cluster_dict["id"] = cluster.id
    clusters.append(cluster_dict)
print(clusters)
# return Response(clusters, status=status.HTTP_200_OK)

1
one
two
three


ValueError: Cannot use None as a query value

In [75]:
print(list(Cluster.objects.all().values_list('name', flat=True)))

['Berambadi', 'Terakanambi', 'Bommanahalli', 'Yadalaya', 'HD Kote', 'Nanjangud', 'Baragi', 'Sargur', 'Talavadi', 'masinakudi']


In [102]:
user_obj = list(UserProfile.objects.filter(user_id__in=list(set(subordinate_user_ids))).values('user_type_id','user_id'))
pandas

[{'user_type_id': 6, 'user_id': 77},
 {'user_type_id': 5, 'user_id': 22},
 {'user_type_id': 5, 'user_id': 23},
 {'user_type_id': 5, 'user_id': 24},
 {'user_type_id': 5, 'user_id': 25},
 {'user_type_id': 5, 'user_id': 26},
 {'user_type_id': 5, 'user_id': 27},
 {'user_type_id': 5, 'user_id': 28},
 {'user_type_id': 5, 'user_id': 29},
 {'user_type_id': 5, 'user_id': 30},
 {'user_type_id': 6, 'user_id': 41},
 {'user_type_id': 6, 'user_id': 107},
 {'user_type_id': 6, 'user_id': 73},
 {'user_type_id': 6, 'user_id': 31},
 {'user_type_id': 6, 'user_id': 71},
 {'user_type_id': 6, 'user_id': 109},
 {'user_type_id': 6, 'user_id': 106},
 {'user_type_id': 6, 'user_id': 43},
 {'user_type_id': 6, 'user_id': 72},
 {'user_type_id': 6, 'user_id': 60},
 {'user_type_id': 6, 'user_id': 36},
 {'user_type_id': 6, 'user_id': 40},
 {'user_type_id': 6, 'user_id': 37},
 {'user_type_id': 6, 'user_id': 39},
 {'user_type_id': 6, 'user_id': 38},
 {'user_type_id': 6, 'user_id': 35},
 {'user_type_id': 6, 'user_id': 85}

In [31]:
procurement_obj = list(ProcurementGroup.objects.filter().values_list('id', 'procurement__id'))
columns_list = ['Group_id', 'Procurement_id']
df = pd.DataFrame(procurement_obj, columns=columns_list).fillna('')
un_mapped = df[df['Procurement_id'] == '']
un_mapped
# df

,Group_id,Procurement_id
4334,619,
4335,382,
4336,853,
4337,1486,
4338,78,


In [55]:
procurement_obj = list(Procurement.objects.all().values_list('id', 'procurement_group_id', 'ticket_number', 'str_weight', 'gunnybag_count', ))
columns_list = ['Procurement_id', 'Procurement_group_id']
df = pd.DataFrame(procurement_obj, columns=columns_list)

df.shape

(4334, 2)

In [77]:
df2 = df[df['Procurement_group_id'].duplicated(keep='first')].sort_values(by=['Procurement_group_id'])
print(df2.shape)
from IPython.core.display import HTML
display(HTML(df2.to_html()))

(102, 2)


,Procurement_id,Procurement_group_id
15,27,15
40,28,19
14,26,20
16,30,20
20,36,23
76,61,31
33,53,33
578,55,36
72,56,36
73,57,36


In [89]:
procurement_values = list(Procurement.objects.all().values_list(
            "id",
            "procurement_group_id",
            "ticket_number",
            "vehicle_number",
            "vehicle_driver_name",
            "str_weight",
            "str_weight_unit__name",
            "remark",
            "empty_vehicle_timestamp",
            "empty_vehicle_weight_data_inputed",
            "empty_vehicle_weight",
            "loaded_vehicle_timestamp",
            "loaded_vehicle_weight_data_inputed",
            "loaded_vehicle_weight",
            "gunnybag_count",
            "gunnybag_weight",
            "moisture",
            "produce_gross_weight",
            "unit__name",
            "procurement_group__agent_id",
            "procurement_group__agent__first_name",
            "procurement_group__procurement_date",
            "procurement_group__procurement_produce__name",
            "procurement_group__procurement_produce__id",
            "procurement_group__season_id",
            "procurement_group__produce_net_weight",
            "procurement_group__price_per_unit",
            "procurement_group__unit_for_pricing",
            "procurement_group__cost",
            "procurement_group__payment_to_wallet",
            "procurement_group__payment_to_agent",
            "procurement_group__agent_price_deduction",
            "have_other_agent_farmer",
        )
    )

procurement_columns = [
        "procurement_id",
        "procurement_group_id",
        "ticket_number",
        "vehicle_number",
        "vehicle_driver_name",
        "str_weight",
        "str_weight_unit_name",
        "remark",
        "empty_vehicle_timestamp",
        "empty_vehicle_weight_data_inputed",
        "empty_vehicle_weight",
        "loaded_vehicle_timestamp",
        "loaded_vehicle_weight_data_inputed",
        "loaded_vehicle_weight",
        "gunnybag_count",
        "gunnybag_weight",
        "moisture",
        "produce_gross_weight",
        "unit_name",
        "agent_id",
        "agent_username",
        "procurement_date",
        "procurement_produce_name",
        "procurement_produce_id",
        "season_id",
        "produce_net_weight",
        "price_per_unit",
        "unit_for_pricing",
        "cost",
        "payment_to_wallet",
        "payment_to_agent",
        "agent_price_deduction",
        "have_other_agent_farmer",
    ]
procurement_df = pd.DataFrame(procurement_values, columns=procurement_columns)

duplicatd_procurement_df = procurement_df[procurement_df['procurement_group_id'].duplicated(keep='first')].sort_values(by=['procurement_group_id'])
duplicatd_procurement_df
# from IPython.core.display import HTML
# display(HTML(duplicatd_procurement_df.to_html()))

,procurement_id,procurement_group_id,ticket_number,vehicle_number,vehicle_driver_name,str_weight,str_weight_unit_name,remark,empty_vehicle_timestamp,empty_vehicle_weight_data_inputed,...,procurement_produce_id,season_id,produce_net_weight,price_per_unit,unit_for_pricing,cost,payment_to_wallet,payment_to_agent,agent_price_deduction,have_other_agent_farmer
15,27,15,2020062501013,KA10A4351,Mallu,2212.00,kg,None,2020-06-25 05:39:01+00:00,MANU,...,1,2,2230.00,8.70,1,19401.00,15610.00,3791.00,7.00,False
40,28,19,2020062501014,KA09A6614,Ravi,2540.00,kg,TOTAL MISTAKE CORRCATION,2020-06-25 06:02:58+00:00,MANU,...,1,2,2690.00,8.70,1,23403.00,4035.00,19368.00,1.50,False
14,26,20,2020062501012,KA10A0921,basavana,985.00,kg,None,2020-06-25 05:24:55+00:00,MANU,...,1,2,995.00,8.70,1,8656.50,1492.50,7164.00,1.50,False
16,30,20,2020062501016,KA10A0548,Srinivas,980.00,kg,None,2020-06-25 06:41:10+00:00,MANU,...,1,2,995.00,8.70,1,8656.50,1492.50,7164.00,1.50,False
20,36,23,2020062501023,KA10A2151,Nagesh,2962.00,kg,None,2020-06-25 08:59:45+00:00,MANU,...,1,2,3027.00,8.70,1,26334.90,4540.50,21794.40,1.50,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3091,3172,3186,2020081401059,KA10A4619,SANTHOSH,2320.00,kg,None,2020-08-14 11:41:40+00:00,Mahesh B R,...,1,2,2350.00,8.70,1,20445.00,4700.00,15745.00,2.00,False
3194,3279,3284,2020081601042,KA10A1859,PRATHAP,2488.00,kg,None,2020-08-16 09:42:35+00:00,Mahesh B R,...,1,2,2549.00,8.70,1,22176.30,5098.00,17078.30,2.00,False
3298,3377,3401,2020081801032,KA105150,PRASAD,2507.00,kg,None,2020-08-18 06:01:13+00:00,Mahesh B R,...,1,2,2548.00,8.70,1,22167.60,0.00,22167.60,0.00,False
3544,3620,3647,2020082401015,KA10A5170,Kemparaju,2176.00,kg,None,2020-08-24 11:01:25+00:00,Mahesh B R,...,1,2,2290.00,8.70,1,19923.00,0.00,19923.00,0.00,False


### Cluster Details For Mobile App

In [2]:
selected_date = "2020-08-10"
date = datetime.datetime.strptime(selected_date, "%Y-%m-%d")

user_id = 78
print("superior id:", user_id)
user_type_id = UserProfile.objects.get(user=user_id).user_type_id
if user_type_id == 5:
    subordinate_user_ids = [user_id]
else:
    subordinate_user_ids = []
    if UserHierarchyMap.objects.filter(superior_id=user_id).exists():
        subordinates = UserHierarchyMap.objects.filter(superior_id=user_id)
        sub_list = list(subordinates.values_list("subordinate", flat=True))
        subordinate_user_ids.extend(sub_list)
        print(subordinate_user_ids)
        if UserHierarchyMap.objects.filter(
            superior_id__in=subordinate_user_ids
        ).exists():
            subordinates = UserHierarchyMap.objects.filter(
                superior_id__in=subordinate_user_ids
            )
            sub_list = list(subordinates.values_list("subordinate", flat=True))
            subordinate_user_ids.extend(sub_list)
            print("two")
            if UserHierarchyMap.objects.filter(
                superior_id__in=subordinate_user_ids
            ).exists():
                subordinates = UserHierarchyMap.objects.filter(
                    superior_id__in=subordinate_user_ids
                )
                sub_list = list(subordinates.values_list("subordinate", flat=True))
                subordinate_user_ids.extend(sub_list)
                print("three")
                if UserHierarchyMap.objects.filter(
                    superior_id__in=subordinate_user_ids
                ).exists():
                    subordinates = UserHierarchyMap.objects.filter(
                        superior_id__in=subordinate_user_ids
                    )
                    sub_list = list(
                        subordinates.values_list("subordinate", flat=True)
                    )
                    subordinate_user_ids.extend(sub_list)
                    print("four")
print(subordinate_user_ids)
subordinate_user_ids = list(set(subordinate_user_ids))
subordinate_user_ids = list(
    UserProfile.objects.filter(
        user_id__in=subordinate_user_ids, user_type_id=5
    ).values_list("user_id", flat=True)
)

users_clusters_ids = list(UserClusterMap.objects.filter(user_id__in=subordinate_user_ids).values_list('cluster_id', flat=True))

sowing_boundary = list(
    SowingBoundary.objects.filter(
        timestamp__date=date, sowing__cultivation_phase_id=2
    ).values_list(
        "id",
        "sowing_id",
        "sowing__area_calculated_via_geo_fencing",
        "sowing__farmer__id",
        "sowing__area",
    )
)
sowing_values = ["id", "sowing_id", "geo_fence_area", "farmer_id", "area"]
sowing_df = pd.DataFrame(sowing_boundary, columns=sowing_values)
sowing_df = sowing_df.drop_duplicates(
    subset="sowing_id", keep="first", inplace=False
)
# farmer cluster
farmer_cluster_values = list(
    FarmerClusterSeasonMap.objects.filter(season_id=2).values_list(
        "id", "farmer_id", "cluster_id", "cluster__name"
    )
)
farmer_cluster_column = ["id", "farmer_id", "cluster_id", "cluster_name"]
farmer_cluster_df = pd.DataFrame(
    farmer_cluster_values, columns=farmer_cluster_column
)

merged_df = pd.merge(
    sowing_df,
    farmer_cluster_df,
    left_on="farmer_id",
    right_on="farmer_id",
    how="left",
)
merged_df = merged_df.fillna(0)
master_list = []
total_dict = {}
total_dict["Cluster"] = "Total"
total_dict["Farmer count"] = 0
total_dict["Farmer TP area"] = 0
total_dict["Gps area"] = 0
total_dict["TP Acres"] = 0
total_dict["Farmer count gps"] = 0
total_dict["Ratio GPS to TP"] = 0

for cluster in Cluster.objects.filter(id__in=users_clusters_ids):
    filtered_rows = merged_df[merged_df["cluster_id"] == cluster.id]
    temp_dict = {}
    # cluster name
    temp_dict["Cluster"] = cluster.name
    # farmer count for gps marked on that date
    temp_dict["Farmer count"] = len(filtered_rows["farmer_id"].unique())
    # area of tp on selected date
    temp_dict["Farmer TP area"] = round(filtered_rows["area"].sum(), 2)
    # fenced area of selected date
    temp_dict["Gps area"] = round(
        merged_df[merged_df["cluster_id"] == cluster.id]["geo_fence_area"].sum(), 2
    )

    empty_rows = filtered_rows[filtered_rows["geo_fence_area"] != 0]
    temp_dict["TP Acres"] = round(
        empty_rows[empty_rows["cluster_id"] == cluster.id]["area"].sum(), 2
    )
    temp_dict["Farmer count gps"] = len(empty_rows["farmer_id"].unique())
    if temp_dict["Gps area"] == 0:
        temp_dict["Ratio GPS to TP"] = 0
    else:
        temp_dict["Ratio GPS to TP"] = round(
            temp_dict["Gps area"] / temp_dict["TP Acres"], 2
        )
    total_dict["Farmer count"] = round(
        total_dict["Farmer count"] + temp_dict["Farmer count"], 2
    )
    total_dict["Farmer TP area"] = round(
        total_dict["Farmer TP area"] + temp_dict["Farmer TP area"], 2
    )
    total_dict["Gps area"] = round(
        total_dict["Gps area"] + temp_dict["Gps area"], 2
    )
    total_dict["TP Acres"] = round(
        total_dict["TP Acres"] + temp_dict["TP Acres"], 2
    )
    total_dict["Farmer count gps"] = round(
        total_dict["Farmer count gps"] + temp_dict["Farmer count gps"], 2
    )
    total_dict["Ratio GPS to TP"] = round(
        total_dict["Ratio GPS to TP"] + temp_dict["Ratio GPS to TP"], 2
    )
    master_list.append(temp_dict)
master_list.append(total_dict)
df = pd.DataFrame(master_list)
df = df.fillna(0)
# initializing excel
writer = pd.ExcelWriter(str("static/media/") + "day_wise.xlsx", engine="xlsxwriter")
# creating excel sheet with name

df.to_excel(writer, sheet_name="Sheet1", startrow=1)

# assigning that sheet to obj
workbook = writer.book
worksheet = writer.sheets["Sheet1"]
merge_format = workbook.add_format(
    {
        "bold": 1,
        "border": 1,
        "align": "center",
        "valign": "vcenter",
        "fg_color": "yellow",
    }
)

date = datetime.datetime.now().date()
# Merge 3 cells.
worksheet.merge_range("A1:G1", "DAYWISE " + str(date), merge_format)

format1 = workbook.add_format({"num_format": "#,##0.00"})

# Set the column width and format.
worksheet.set_column("B:B", 18, format1)
worksheet.set_column(0, 7, 20)

# Add a header format.
header_format = workbook.add_format({"fg_color": "#D7E4BC"})

# Write the column headers with the defined format.
for col_num, value in enumerate(df.columns.values):
    worksheet.write(0, col_num + 1, value, header_format)
writer.save()
document = {}
try:
    image_path = str("static/media/") + "day_wise.xlsx"
    with open(image_path, "rb") as image_file:
        encoded_image = b64encode(image_file.read())
        document["excel"] = encoded_image
except Exception as err:
    print(err)
document["data"] = df.to_dict("r")
df
# return Response(document, status=status.HTTP_200_OK)

superior id: 78
[79]
two
three
four
[79, 80, 92, 82, 80, 92, 82, 22, 23, 24, 25, 26, 27, 28, 29, 30, 80, 92, 80, 92, 82, 22, 23, 24, 25, 26, 27, 28, 29, 30, 80, 92, 40, 37, 39, 31, 57, 35, 34, 36, 38, 77, 88, 41, 32, 111, 121, 71, 72, 73, 44, 43, 48, 46, 106, 104, 110, 58, 60, 59, 56, 53, 47, 103, 120, 54, 65, 64, 90, 62, 109, 112, 122, 51, 55, 74, 49, 50, 86, 89, 107, 117, 118, 91, 113, 114, 63, 85, 124, 125, 126, 123, 87]
name 'b64encode' is not defined


,Cluster,Farmer count,Farmer TP area,Gps area,TP Acres,Farmer count gps,Ratio GPS to TP
0,Berambadi,18,20.0,14.91,20.0,18,0.75
1,Terakanambi,0,0.0,0.00,0.0,0,0.00
2,Bommanahalli,0,0.0,0.00,0.0,0,0.00
3,Yadalaya,8,7.0,5.79,7.0,8,0.83
4,HD Kote,5,4.0,2.40,4.0,5,0.60
5,Nanjangud,0,0.0,0.00,0.0,0,0.00
6,Baragi,1,0.5,0.40,0.5,1,0.80
7,Sargur,0,0.0,0.00,0.0,0,0.00
8,Talavadi,0,0.0,0.00,0.0,0,0.00
9,masinakudi,0,0.0,0.00,0.0,0,0.00


In [7]:
df_val = [[1,4,5,6,7,8],[1,4,5,6,6,8],[1,4,5,6,5,8],[1,4,5,6,4,8],[1,4,5,6,3,8],[1,4,5,6,2,8]]
col = ['a','b','c','d','e','f']
df = pd.DataFrame(df_val, columns=col)
df['a'] = 'total'
df['total'] = df['e'] + df['f']
df

,a,b,c,d,e,f,total
0,total,4,5,6,7,8,15
1,total,4,5,6,6,8,14
2,total,4,5,6,5,8,13
3,total,4,5,6,4,8,12
4,total,4,5,6,3,8,11
5,total,4,5,6,2,8,10


In [16]:
Harvest.objects.filter(sowing__farmer__code="20BAS055").aggregate(Sum('value'))

{'value__sum': 2603}

### crop_age_based_report

In [4]:
# def serve_crop_age_based_report(request):
# farmer
# cluster
data = {
    "cluster_ids" : [5,10]
}
if len(data["cluster_ids"]) == 0:
    cluster_ids = list(Cluster.objects.all().values_list("id", flat=True))
else:
    cluster_ids = list(
        Cluster.objects.filter(id__in=data["cluster_ids"]).values_list(
            "id", flat=True
        )
    )
    print("cluster filter")
# super_visor
supervisor_ids = list(
    UserProfile.objects.filter(user_type_id=5).values_list("user_id", flat=True)
)

# # agent
# if len(request.data["agent_ids"]) == 0:
#     agent_ids = list(
#         UserProfile.objects.filter(user_type_id=6).values_list("user_id", flat=True)
#     )
# else:
#     print("agent filter")
#     agent_ids = list(
#         UserProfile.objects.filter(
#             user_type_id=6, user_id__in=request.data["agent_ids"]
#         ).values_list("user_id", flat=True)
#     )

#  farmer cluster
cluster_based_farmers_ids = list(
    FarmerClusterSeasonMap.objects.filter(
        season_id=2, cluster_id__in=cluster_ids
    ).values_list("farmer_id", flat=True)
)

# supervisor_based
user_farmer_ids = list(
    UserFarmerMap.objects.filter(
        officer_id__in=supervisor_ids,
        farmer__season_id=2,
        farmer__farmer__id__in=cluster_based_farmers_ids,
    ).values_list("farmer__farmer_id", flat=True)
)

# agent farmer map
farmer_ids = list(
    AgentFarmerMap.objects.filter(
        farmer__season_id=2,
        farmer__farmer__id__in=user_farmer_ids,
    ).values_list("farmer__farmer_id", flat=True)
)

# farmer_ids = Harvest.objects.filter(sowing__season_id=2).values_list('sowing__farmer__id', flat=True)
filtered_farmer_ids = list(
    FarmerClusterSeasonMap.objects.filter(
        season_id=2, farmer_id__in=farmer_ids
    ).values_list("farmer__id", flat=True)
)

sowing_values = list(
    Sowing.objects.filter(
        farmer_id__in=filtered_farmer_ids, season_id=2, cultivation_phase_id=2
    ).values_list(
        "id",
        "farmer_id",
        "sowing_date",
        "area",
        "cultivation_phase_id",
        "cultivation_phase__name",
    )
)
sowing_columns = [
    "sowing_id",
    "farmer_id",
    "sowing_date",
    "area",
    "cultivation_phase_id",
    "cultivation_phase_name",
]
sowing_df = pd.DataFrame(sowing_values, columns=sowing_columns)
today = datetime.datetime.now().date()

sowing_df["crop_age"] = today - sowing_df["sowing_date"]
sowing_df["crop_age"] = sowing_df["crop_age"].astype("timedelta64[D]")
sowing_df["crop_age"] = sowing_df["crop_age"].astype(int)
range_list = [
    "0-54",
    "55-61",
    "62-68",
    "69-75",
    "76-82",
    "83-89",
    "90-96",
    "97-103",
    "104-110",
    "111-117",
    "118-124",
]
master_dict = {}
total = 0
for row in range_list:
    above_sowing_df = sowing_df[(sowing_df["crop_age"] >= int(row.split("-")[0]))]
    below_sowing_df = above_sowing_df[
        (above_sowing_df["crop_age"] <= int(row.split("-")[1]))
    ]
    master_dict[row] = below_sowing_df["area"].sum()
    total = total + below_sowing_df["area"].sum()
above_sowing_df = sowing_df[(sowing_df["crop_age"] >= int(row.split("-")[0]))]
master_dict["125 +"] = above_sowing_df["area"].sum()
master_dict["total"] = total + above_sowing_df["area"].sum()
df = pd.DataFrame(
    list(master_dict.items()), columns=["Age range in days", "Total Area"]
)
# df =  pd.DataFrame(master_dict)

# initializing excel
writer = pd.ExcelWriter(
    str("static/media/") + "sowing_age_wise_report.xlsx", engine="xlsxwriter"
)
# creating excel sheet with name

df.to_excel(writer, sheet_name="Sheet1", startrow=1)

# assigning that sheet to obj
workbook = writer.book
worksheet = writer.sheets["Sheet1"]
merge_format = workbook.add_format(
    {
        "bold": 1,
        "border": 1,
        "align": "center",
        "valign": "vcenter",
        "fg_color": "yellow",
    }
)

date = datetime.datetime.now().date()
# Merge 3 cells.
worksheet.merge_range("A1:C1", "Sowing Age wise report", merge_format)

format1 = workbook.add_format({"num_format": "#,##0.00"})

# Set the column width and format.
worksheet.set_column("B:B", 18, format1)
worksheet.set_column(0, 7, 20)

# Add a header format.
header_format = workbook.add_format({"fg_color": "#D7E4BC"})

# Write the column headers with the defined format.
for col_num, value in enumerate(df.columns.values):
    worksheet.write(0, col_num + 1, value, header_format)
writer.save()
document = {}
try:
    image_path = str("static/media/") + "sowing_age_wise_report.xlsx"
    with open(image_path, "rb") as image_file:
        encoded_image = b64encode(image_file.read())
        document["excel"] = encoded_image
except Exception as err:
    print(err)
document["data"] = master_dict
df['total'] = df[]
# return Response(data=document, status=status.HTTP_200_OK)

cluster filter
name 'b64encode' is not defined


,Age range in days,Total Area
0,0-54,0.0
1,55-61,0.0
2,62-68,0.0
3,69-75,1.0
4,76-82,0.0
5,83-89,0.0
6,90-96,0.0
7,97-103,7.0
8,104-110,5.5
9,111-117,5.0


### Harvest Report

In [7]:
harvest_days_group = {
    1: {'min': 0, 'max': 61},
    2: {'min': 62, 'max': 68},
    3: {'min': 69, 'max': 75},
    4: {'min': 76, 'max': 82},
    5: {'min': 83, 'max': 89},
    6: {'min': 90, 'max': 96},
    7: {'min': 97, 'max': 103},
    8: {'min': 104, 'max': 110},
    9: {'min': 111, 'max': 117},
    10: {'min': 118, 'max': 200},
}
df = pd.DataFrame(harvest_days_group)
df

,1,2,3,4,5,6,7,8,9,10
min,0,62,69,76,83,90,97,104,111,118
max,61,68,75,82,89,96,103,110,117,200


In [4]:
t1 = datetime.datetime.now()
print(t1)
harvest_values = Harvest.objects.filter(sowing__cultivation_phase_id=2, sowing__season_id=2).order_by('sowing_id','date_of_harvest').values_list(
    'sowing_id', 'sowing__farmer_id', 'date_of_harvest', 'value', 'unit__name', 'sowing__sowing_date')
harvest_columns = ['sowing_id', 'farmer_id', 'date_of_harvest', 'value', 'unit', 'sowing_date']
harvest_df = pd.DataFrame(list(harvest_values), columns=harvest_columns)
harvest_df['harvest_days'] = (harvest_df['date_of_harvest'] - harvest_df['sowing_date']).dt.days
harvest_df.loc[harvest_df.harvest_days  < 0, "harvest_days"] = 55
harvest_days = HarvestLevel.objects.filter().order_by('ordinal').values_list('harvest_interval_duration_in_days', flat=True)

harvest_days_group = {
    1: {'min': 0, 'max': 61},
    2: {'min': 62, 'max': 68},
    3: {'min': 69, 'max': 75},
    4: {'min': 76, 'max': 82},
    5: {'min': 83, 'max': 89},
    6: {'min': 90, 'max': 96},
    7: {'min': 97, 'max': 103},
    8: {'min': 104, 'max': 110},
    9: {'min': 111, 'max': 117},
    10: {'min': 118, 'max': 200},
}


harvest_count_dict = {
    1: ('First Harvest Date', '1st Kg'),
    2: ('2nd Harvest Date', '2nd Kg'),
    3: ('3rd Harvest Date', '3rd Kg'),
    4: ('4th Harvest Date', '4th Kg'),
    5: ('5th Harvest Date', '5th Kg'),
    6: ('6th Harvest Date', '6th Kg'),
    7: ('7th Harvest Date', '7th Kg'),
    8: ('8th Harvest Date', '8th Kg'),
    9: ('9th Harvest Date', '9th Kg'),
    10: ('10th Harvest Date', '10th Kg'),
}

harvest_df = harvest_df.assign(**{'harvest_count': None})

for i in harvest_days_group:
    harvest_df.loc[(harvest_df['harvest_days'] >= harvest_days_group[i]['min']) & (harvest_df['harvest_days'] <= harvest_days_group[i]['max']), 'harvest_count'] = i

for sowing_id in harvest_df['sowing_id'].unique():
    for harvest_count in harvest_days_group:
        temp_df = harvest_df[(harvest_df['sowing_id'] == sowing_id) & (harvest_df['harvest_count'] == harvest_count)]
        # print(temp_df)
        if not temp_df.empty:
            # print('{} = {} = {}'.format(sowing_id, harvest_count, temp_df.groupby('harvest_count')['value'].sum().iat[0]))
            sum_of_harvest = temp_df.groupby('harvest_count')['value'].sum().iat[0]
            # print(sum_of_harvest)
            harvest_df.loc[harvest_df['sowing_id'] == sowing_id, harvest_count_dict[harvest_count][1]] = sum_of_harvest
            temp_harvest_date = temp_df['date_of_harvest'].iat[0]
            # print(temp_harvest_date)
            harvest_df.loc[harvest_df['sowing_id'] == sowing_id, harvest_count_dict[harvest_count][0]] = temp_harvest_date

harvest_df.head()

harvest_df = harvest_df.drop_duplicates(subset=['sowing_id','harvest_count'], keep='first')

farmer_values = Farmer.objects.filter(id__in=harvest_df['farmer_id']).values_list('id', 'first_name', 'last_name',
                                                                                'village', 'code')
farmer_columns = ['id', 'farmer_first_name', 'farmer_last_name', 'farmer_village', 'farmer_code']
farmer_df = pd.DataFrame(list(farmer_values), columns=farmer_columns)

agent_values = AgentFarmerMap.objects.filter(farmer__farmer_id__in=harvest_df['farmer_id']).values_list('farmer_id',
                                                                                                        'agent_id',
                                                                                                        'agent__first_name',
                                                                                                        'agent__last_name')

agent_columns = ['farmer_id', 'agent_id', 'agent_first_name', 'agent_last_name']

agent_df = pd.DataFrame(list(agent_values), columns=agent_columns)

harvest_df = harvest_df.merge(farmer_df, how='left', left_on='farmer_id', right_on='id').merge(agent_df, how='left',left_on='farmer_id',right_on='farmer_id')
# print(harvest_df.columns)
harvest_df = harvest_df.fillna(0)

harvest_table_list = list(
Harvest.objects.filter(sowing__cultivation_phase_id=2).values_list(
        "sowing__farmer__code",
        "sowing__farmer__id",
        "sowing__farmer__first_name",
        "sowing__farmer__last_name",
        "sowing__farmer__village__name",
        "sowing__season__name",
        "value",
        "date_of_harvest",
        "sowing__area",
        "sowing__farmer__cluster__name",
    )
)
column_list = [
    "farmer_code",
    "farmer_id",
    "farmer_first_name",
    "farmer_last_name",
    "farmer_village",
    "season",
    "yeald",
    "date_of_harvest",
    "area",
    "cluster_name",
]
harvest_data_frame = pd.DataFrame(harvest_table_list, columns=column_list)

# agent_data
agent_farmer_list = list(
    AgentFarmerMap.objects.filter(
        farmer__farmer__code__in=harvest_data_frame["farmer_code"]
    ).values_list("farmer__farmer__code", "agent__first_name")
)
agent_farmer_columns = ["farmer_code", "agent_name"]
agent_farmer_df = pd.DataFrame(agent_farmer_list, columns=agent_farmer_columns)

# supervisor_data
supervisor_farmer_list = list(
    UserFarmerMap.objects.filter(
        farmer__farmer__code__in=harvest_data_frame["farmer_code"]
    ).values_list("farmer__farmer__code", "officer__username")
)
supervisor_farmer_colums = ["farmer_code", "supervisor_name"]
supervisor_farmer_df = pd.DataFrame(
    supervisor_farmer_list, columns=supervisor_farmer_colums
)

# merged_data
farmer_agent_map_df = pd.merge(
    harvest_data_frame,
    agent_farmer_df,
    left_on="farmer_code",
    right_on="farmer_code",
)
supervisor_fatmer_agent_map_df = pd.merge(
    farmer_agent_map_df,
    supervisor_farmer_df,
    left_on="farmer_code",
    right_on="farmer_code",
)

# sum yeald and find last harvest date finally merge with supervisor_fatmer_agent_map_df
grouped_df = (
    supervisor_fatmer_agent_map_df.groupby(["farmer_code"])
    .agg({"yeald": "sum", "date_of_harvest": "max"})
    .rename(
        columns={"yeald": "total_yeild", "date_of_harvest": "last_harvest_date"}
    )
)
final_supervisor_fatmer_agent_map_df = pd.merge(
    supervisor_fatmer_agent_map_df,
    grouped_df,
    left_on="farmer_code",
    right_index=True,
)
final_supervisor_fatmer_agent_map_df = final_supervisor_fatmer_agent_map_df.drop(
    columns=["yeald", "date_of_harvest"]
)
final_supervisor_fatmer_agent_map_df = (
    final_supervisor_fatmer_agent_map_df.drop_duplicates()
)

# final_df
cons_df = final_supervisor_fatmer_agent_map_df.reindex(
    columns=[
        "farmer_id",
        "cluster_name",
        "last_harvest_date",
        "area",
        "total_yeild",
    ]
)

harvest_df = harvest_df.fillna(0)
harvest_df['total'] = harvest_df['1st Kg'] + harvest_df['2nd Kg'] + harvest_df['3rd Kg']+ harvest_df['4th Kg']+ harvest_df['5th Kg']+ harvest_df['6th Kg']+ harvest_df['7th Kg']+ harvest_df['8th Kg']+ harvest_df['9th Kg']+ harvest_df['10th Kg']
df = pd.merge(harvest_df, cons_df, left_on="farmer_id", right_on="farmer_id", how="left")
df.drop_duplicates(subset="farmer_id", keep="first", inplace=True)
df = df.drop(['sowing_id', 'farmer_id', 'unit', 'value', 'date_of_harvest'], axis=1)
df = df.fillna(0)

2020-10-23 05:28:44.793343


In [3]:
# df = df[['farmer_first_name', 'farmer_last_name', 'farmer_code', 'agent_first_name', 'agent_last_name',
#         'First Harvest Date', '1st Kg',
#         '2nd Harvest Date', '2nd Kg', '3rd Harvest Date', '3rd Kg', '4th Harvest Date', '4th Kg',
#         '5th Harvest Date', '5th Kg',
#         '6th Harvest Date', '6th Kg', '7th Harvest Date', '7th Kg', '8th Harvest Date', '8th Kg',
#         '9th Harvest Date', '9th Kg',
#         '10th Harvest Date', '10th Kg', 'total', 'total_yeild', 'area']]

# initializing excel
writer = pd.ExcelWriter(str('static/media/') + 'harvest_wise_reports.xlsx', engine='xlsxwriter')

# creating excel sheet with name
df1 = df[df['total'] == df['total_yeild']]
df1['Sl.no'] = range(1, 1+len(df1))
df1 = df1[['Sl.no', 'supervisor_name', 'farmer_first_name', 'farmer_last_name', 'farmer_code', 'agent_first_name', 'agent_last_name',
        'First Harvest Date', '1st Kg',
        '2nd Harvest Date', '2nd Kg', '3rd Harvest Date', '3rd Kg', '4th Harvest Date', '4th Kg',
        '5th Harvest Date', '5th Kg',
        '6th Harvest Date', '6th Kg', '7th Harvest Date', '7th Kg', '8th Harvest Date', '8th Kg',
        '9th Harvest Date', '9th Kg',
        '10th Harvest Date', '10th Kg', 'total', 'total_yeild', 'area']]


df2 = df[df['total'] != df['total_yeild']]
df2['Sl.no'] = range(1, 1+len(df2))
df2 = df2[['Sl.no', 'supervisor_name', 'farmer_first_name', 'farmer_last_name', 'farmer_code', 'agent_first_name', 'agent_last_name',
        'First Harvest Date', '1st Kg',
        '2nd Harvest Date', '2nd Kg', '3rd Harvest Date', '3rd Kg', '4th Harvest Date', '4th Kg',
        '5th Harvest Date', '5th Kg',
        '6th Harvest Date', '6th Kg', '7th Harvest Date', '7th Kg', '8th Harvest Date', '8th Kg',
        '9th Harvest Date', '9th Kg',
        '10th Harvest Date', '10th Kg', 'total', 'total_yeild', 'area']]


df1.to_excel(writer, sheet_name='Sheet1', index=False, startrow=1)
df2.to_excel(writer, sheet_name='Sheet2', index=False, startrow=1)

# assigning that sheet to obj
workbook = writer.book
worksheet1 = writer.sheets['Sheet1']
worksheet2 = writer.sheets['Sheet2']
merge_format = workbook.add_format({
    'bold': 1,
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'fg_color': 'yellow'})

# Merge 3 cells.
format = "%Y-%m-%d %H:%M:%S %Z%z"
now_utc = datetime.datetime.now(timezone('UTC'))
now_asia = now_utc.astimezone(timezone('Asia/Kolkata'))
current_time = now_asia.strftime("%b %d %Y %H:%M:%S")

name = 'Harvest Range wise report' + ' (Updated on: ' + str(current_time) +')'
worksheet1.merge_range('A1:AC1', name, merge_format)
worksheet2.merge_range('A1:AC1', 'Late harvest data (some entries will not be included for these entries)', merge_format)
format1 = workbook.add_format({'num_format': '#,##0.00'})

# Set the column width and format.
worksheet1.set_column('B:B', 18, format1)
worksheet1.set_column(0, 28, 20)

# Set the column width and format.
worksheet2.set_column('B:B', 18, format1)
worksheet2.set_column(0, 28, 20)

# Add a header format.
header_format = workbook.add_format({'fg_color': '#D7E4BC'})

# Write the column headers with the defined format.
for col_num, value in enumerate(df1.columns.values):
    worksheet1.write(0, col_num + 1, value, header_format)

# Write the column headers with the defined format.
for col_num, value in enumerate(df2.columns.values):
    worksheet2.write(0, col_num + 1, value, header_format)

writer.save()
t2 = datetime.datetime.now()
print(t2 - t1)

NameError: name 'df' is not defined

In [11]:
agent_farmer_map_list = list(AgentFarmerMap.objects.filter(farmer__season_id=2).order_by('agent_id').values_list('agent_id', 'agent__first_name', 'farmer__farmer__code', 'farmer__farmer__sowing__cultivation_phase', 'farmer__farmer__sowing__area'))
agent_farmer_columns = ['agent_id', 'agent_name', 'farmer_code', 'cultivation_phase', 'area']
agent_farmer_df = pd.DataFrame(agent_farmer_map_list, columns=agent_farmer_columns).drop_duplicates()
                            
#seprating_cultivation_phase
df_pase1 = agent_farmer_df[agent_farmer_df['cultivation_phase']==1]
df_pase2 = agent_farmer_df[agent_farmer_df['cultivation_phase']==2]
                            
farmer_cultivetion_phase_df = pd.merge(df_pase1, df_pase2, left_on='farmer_code', right_on='farmer_code', how='left')
farmer_cultivetion_phase_df = farmer_cultivetion_phase_df.rename(columns={'area_x':'naursery', 'area_y':'tp'})
farmer_cultivetion_phase_df = farmer_cultivetion_phase_df.fillna(0)
farmer_cultivetion_phase_df["total_area"] = farmer_cultivetion_phase_df["naursery"] + farmer_cultivetion_phase_df["tp"]

farmer_cultivetion_phase_df = farmer_cultivetion_phase_df.drop(columns=['agent_name_y', 'agent_id_y', 'cultivation_phase_x', 'cultivation_phase_y', ])

grouped_df = farmer_cultivetion_phase_df.groupby(['agent_id_x','agent_name_x']).agg({'farmer_code': 'unique', 'naursery': 'sum', 'tp': 'sum', 'total_area': 'sum'})
grouped_df['farmer_count'] = grouped_df['farmer_code'].str.len().astype(float)
grouped_df = grouped_df.reset_index().reindex(columns=['agent_name_x', 'farmer_count', 'naursery', 'tp', 'total_area']).rename(columns={'agent_name_x': 'agent_name', 'naursery': 'sowing'})
df = grouped_df

# initializing excel
writer = pd.ExcelWriter(
    str("static/media/") + "agnet_wise_cultivation_report.xlsx", engine="xlsxwriter")
# creating excel sheet with name

df.to_excel(writer, sheet_name="Sheet1", startrow=1, index=False)

# assigning that sheet to obj
workbook = writer.book
worksheet = writer.sheets["Sheet1"]
merge_format = workbook.add_format(
    {
        "bold": 1,
        "border": 1,
        "align": "center",
        "valign": "vcenter",
        "fg_color": "yellow",
    }
)

date = datetime.datetime.now().date()
# Merge 3 cells.
worksheet.merge_range("A1:F1", "Agent Wise Cultivation Report", merge_format)

# format1 = workbook.add_format({"num_format": "#,##0.00"})

# Set the column width and format.
# worksheet.set_column("B:B", 18, format1)
worksheet.set_column(0, 7, 20)

# Add a header format.
header_format = workbook.add_format({"fg_color": "#D7E4BC"})

# Write the column headers with the defined format.
for col_num, value in enumerate(df.columns.values):
    worksheet.write(0, col_num + 1, value, header_format)
writer.save()
document = {}
try:
    image_path = str("static/media/") + "sowing_age_wise_report.xlsx"
    with open(image_path, "rb") as image_file:
        encoded_image = b64encode(image_file.read())
        document["excel"] = encoded_image
except Exception as err:
    print(err)
# document["data"] = master_dict

name 'b64encode' is not defined


In [36]:
farmer_by_agent = agent_farmer_df.groupby('agent_id')['farmer_code'].unique()
for i in farmer_by_agent:
    print(len(i))

28
79
38
96
3
111
8
85
63
156
67
58
39
54
121
34
70
1
35
90
36
110
83
2
47
57
162
192
206
231
75
55
22
45
12
30
69
22
37
13
43
3
8
44
47
18
25
100
76
74
20
6
12


In [98]:
unique_fields = ['farmer_id']
duplicates = (AgentFarmerMap.objects.values(*unique_fields).order_by().annotate(max_id=Max('id'), count_id=Count('id')).filter(count_id__gt=1))
duplicates

<QuerySet []>

In [19]:


farmer_cultivetion_phase_df = farmer_cultivetion_phase_df.rename(columns={'cultivation_phase_x':'pase1', 'cultivation_phase_y':'pase2', 'area_x':'naursery', 'area_y':'tp'})
farmer_cultivetion_phase_df = farmer_cultivetion_phase_df.fillna(0)
farmer_cultivetion_phase_df["total_area"] = farmer_cultivetion_phase_df["naursery"] + farmer_cultivetion_phase_df["tp"]
farmer_cultivetion_phase_df = farmer_cultivetion_phase_df.drop_duplicates()
farmer_cultivetion_phase_df

,farmer_code,pase1,naursery,pase2,tp,total_area
0,20BPM001,1,1.0,0.0,0.0,1.0
1,19NMN104,1,0.8,0.0,0.0,0.8
2,20YMG004,1,1.0,2.0,1.0,2.0
3,19NPM232,1,0.5,0.0,0.0,0.5
4,20BAM001,1,1.5,2.0,1.5,3.0
...,...,...,...,...,...,...
2917,20NSH016,1,1.0,2.0,1.0,2.0
2918,20YMG082,1,1.0,2.0,1.0,2.0
2919,20YMG112,1,1.0,2.0,1.0,2.0
2920,20NSH030,1,1.0,2.0,1.0,2.0


In [18]:
agent_former_area_df = pd.merge(agent_farmer_df, farmer_cultivetion_phase_df, left_on='farmer_code', right_on='farmer_code', how='left')
agent_former_area_df = agent_former_area_df.fillna(0)
agent_former_area_df = agent_former_area_df.drop_duplicates()
# agent_former_area_df
grouped_df = agent_former_area_df.groupby(['agent_id']).agg({'farmer_code': 'count', 'naursery': 'sum', 'tp': 'sum', 'total_area': 'sum'})#.rename(columns={'naursery': 'naursery_area', 'tp': 'tp_area', 'total_area': 'total_area'})
grouped_df[grouped_df.index == 106]

,farmer_code,naursery,tp,total_area
agent_id,,,,
106,43,26.0,26.0,52.0


In [38]:
len(list(AgentFarmerMap.objects.filter(farmer__season_id=2,agent_id=37).values_list('farmer_id')))

111

### Geo fence

In [14]:
# request_data
data_dict = {
    'selected_cluster_ids': [],
    'selected_afs_ids': [],
    'selected_date': None,
    'selected_data_source_ids': [],
    'agent_ids': [31],
}

# data_dict = request.data

sowing_geo_fence_map_obj = SowingBoundaryMap.objects.filter()

if len(data_dict["selected_afs_ids"]) != 0:
    print("hi")
    sowing_geo_fence_map_obj = sowing_geo_fence_map_obj.filter(sowing__farmer__created_by_id__in=data_dict["selected_afs_ids"])

if len(data_dict["selected_data_source_ids"]) != 0:
    sowing_geo_fence_map_obj = sowing_geo_fence_map_obj.filter(data_source_id__in=data_dict["selected_data_source_ids"])

if len(data_dict["agent_ids"]) != 0:
    sowing_geo_fence_map_obj = sowing_geo_fence_map_obj.filter(sowing__farmer__farmerclusterseasonmap__agentfarmermap__agent_id__in=data_dict["agent_ids"])

sowing_boundry_ids = sowing_geo_fence_map_obj.values_list("sowing_boundry", flat=True)
field_ids = sowing_geo_fence_map_obj.values_list("sowing_id", flat=True)

farmer_ids = list(set(list(sowing_geo_fence_map_obj.values_list("sowing__farmer_id", flat=True))))

sowing_geo_fence_map_list = list(
    sowing_geo_fence_map_obj.values_list( "sowing__farmer_id", "sowing_id", "sowing__area_calculated_via_geo_fencing", "data_source", "data_source__name", "sowing__farmer__first_name", "sowing__farmer__last_name", "sowing__farmer__village__name", "sowing__farmer__created_by__username", "sowing__farmer__created_by_id")
)
sowing_geo_fence_map_column = [ "farmer_id", "sowing_id", "area_in_acre", "data_source", "data_source_name", "first_name", "last_name", "farmer_village", "created_by_name", "created_by_id",
]
sowing_geo_fence_map_df = pd.DataFrame(sowing_geo_fence_map_list, columns=sowing_geo_fence_map_column)

if data_dict["selected_cluster_ids"]:
    print("cluster filter applied")
    farmer_cluster_season_map = FarmerClusterSeasonMap.objects.filter(farmer_id__in=farmer_ids, cluster_id__in=request.data["selected_cluster_ids"],)
else:
    farmer_cluster_season_map = FarmerClusterSeasonMap.objects.filter(farmer_id__in=farmer_ids)

cluster_farmer_list = list(farmer_cluster_season_map.values_list("farmer_id", "cluster_id", "cluster__name"))
cluster_farmer_columns = ["farmer_id", "cluster_id", "cluster_name"]
cluster_df = pd.DataFrame(cluster_farmer_list, columns=cluster_farmer_columns)

farmer_cluster_season_map_df = pd.merge(sowing_geo_fence_map_df, cluster_df, how="left")

agent_farmer_map = list(AgentFarmerMap.objects.filter(farmer__in=farmer_cluster_season_map).values_list('farmer__farmer_id', 'agent__first_name', 'agent_id'))
agent_columns = ['farmer_id', 'agent_name', 'agent_id']
agent_df = pd.DataFrame(agent_farmer_map, columns=agent_columns)

farmer_cluster_season_map_df = pd.merge(farmer_cluster_season_map_df, agent_df, left_on="farmer_id", right_on="farmer_id", how="left")

sowing_boundry_obj = SowingBoundary.objects.filter(id__in=sowing_boundry_ids).order_by("id")

sowing_boundry_list = list(sowing_boundry_obj.values_list("sowing_id", "latitude", "longitude"))
sowing_boundry_column = ["sowing_id", "lat", "lng"]
sowing_boundry_df = pd.DataFrame(sowing_boundry_list, columns=sowing_boundry_column)

if not sowing_boundry_df.empty:
    sowing_boundry_series = (sowing_boundry_df.groupby("sowing_id").apply(lambda x: x.to_dict("r")).to_frame())
else:
    sowing_boundry_series = sowing_boundry_df

final_df = farmer_cluster_season_map_df.merge(sowing_boundry_series, how="left", left_on="sowing_id", right_on="sowing_id")

if len(data_dict["selected_cluster_ids"]) != 0:
    print("cluster filter applied")
    final_df = final_df[
        final_df["cluster_id"].isin(request.data["selected_cluster_ids"])
    ]

final_df["fillColor"] = "#14b806"
final_df["strokeColor"] = "#14b806"

# # fill color / bg color
# filtered_clusters = request.data["bg_color_schemes"].keys()
# for index, cluster in enumerate(filtered_clusters):
#     print(cluster)
#     final_df.loc[final_df.cluster_name == cluster, "fillColor"] = request.data[
#         "bg_color_schemes"
#     ][cluster]

# # fill color / bg color
# filtered_source = request.data["border_color_schemes"].keys()
# for index, source in enumerate(filtered_source):
#     print(source)
#     final_df.loc[final_df.data_source_name == source, "strokeColor"] = request.data["border_color_schemes"][source]
#     # color_list = ['#e8505b', '#f9d56e', '#f3ecc2', '#14b1ab']
# final_df.rename(columns={0: "paths"}, inplace=True)
final_df = final_df.fillna(0)

data_dict = {
    "fence_data": final_df.to_dict("r"),
    "total_area": final_df["area_in_acre"].sum(),
    "is_data_available": final_df.empty,
}
final_df

,farmer_id,sowing_id,area_in_acre,data_source,data_source_name,first_name,last_name,farmer_village,created_by_name,created_by_id,cluster_id,cluster_name,agent_name,agent_id,0,fillColor,strokeColor
0,3354,2915,0.902,1,2019 Marigold Farmers,Ebrahim,hydrahali,Bheemanabeedu,ajith,22,1,Berambadi,Arasashetty,31,"[{'sowing_id': 2915, 'lat': 11.7791180, 'lng':...",#14b806,#14b806
1,3356,1663,0.276,1,2019 Marigold Farmers,krisnha,gopalshetty,Bheemanabeedu,ajith,22,1,Berambadi,Arasashetty,31,"[{'sowing_id': 1663, 'lat': 11.7865130, 'lng':...",#14b806,#14b806
2,3356,3064,0.305,1,2019 Marigold Farmers,krisnha,gopalshetty,Bheemanabeedu,ajith,22,1,Berambadi,Arasashetty,31,"[{'sowing_id': 3064, 'lat': 11.7866150, 'lng':...",#14b806,#14b806
3,3357,2917,1.187,1,2019 Marigold Farmers,javarashetty,siddashetty,Bheemanabeedu,ajith,22,1,Berambadi,Arasashetty,31,"[{'sowing_id': 2917, 'lat': 11.7961580, 'lng':...",#14b806,#14b806
4,3358,2918,0.687,1,2019 Marigold Farmers,malishetty,chittirshetty,Bheemanabeedu,ajith,22,1,Berambadi,Arasashetty,31,"[{'sowing_id': 2918, 'lat': 11.7923790, 'lng':...",#14b806,#14b806
5,3359,2919,0.455,1,2019 Marigold Farmers,gokulshetty,bellashetty,Bheemanabeedu,ajith,22,1,Berambadi,Arasashetty,31,"[{'sowing_id': 2919, 'lat': 11.7841000, 'lng':...",#14b806,#14b806
6,3359,3058,0.507,1,2019 Marigold Farmers,gokulshetty,bellashetty,Bheemanabeedu,ajith,22,1,Berambadi,Arasashetty,31,"[{'sowing_id': 3058, 'lat': 11.7991430, 'lng':...",#14b806,#14b806
7,3360,2920,0.345,1,2019 Marigold Farmers,channabasavashetty,chennashetty,Bheemanabeedu,ajith,22,1,Berambadi,Arasashetty,31,"[{'sowing_id': 2920, 'lat': 11.7862910, 'lng':...",#14b806,#14b806
8,3361,2921,0.531,1,2019 Marigold Farmers,gopalshetty,chikkagopalshetty,Bheemanabeedu,ajith,22,1,Berambadi,Arasashetty,31,"[{'sowing_id': 2921, 'lat': 11.7921320, 'lng':...",#14b806,#14b806
9,3363,2940,0.535,1,2019 Marigold Farmers,basavarajun,gokulshetty,Bheemanabeedu,ajith,22,1,Berambadi,Arasashetty,31,"[{'sowing_id': 2940, 'lat': 11.7744170, 'lng':...",#14b806,#14b806


#  Back to ccgb Agent Comnoissuerequest

/Users/hari/.pyenv/versions/3.8.6/lib/python3.8/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [6]:
# Agent details dict   
import pandas as pd
issue_request_ids = [20,19]
agent_id=51
combo_issue_agent_obj = ComboIssueRequestAgentMap.objects.filter(combo_issue_request_id__in=issue_request_ids, agent_id=agent_id, agent__userclustermap__season_id=2)  
agent_details_list = list(combo_issue_agent_obj.values_list('agent__first_name', 'agent__userprofile__code', 'agent__userprofile__village__name', 'agent__userclustermap__cluster__name', 'agent__subordinate_user__superior__first_name'))
data_dict = pd.DataFrame(agent_details_list, columns=['agent_name', 'agent_code', 'village_ame', 'cluster_name', 'superior_name']).drop_duplicates().to_dict('r')[0]
data_dict

{'agent_name': 'C Mahesha',
 'agent_code': 'AG021',
 'village_ame': 'Depapura',
 'cluster_name': 'Bommanahalli',
 'superior_name': 'AFS06'}

In [4]:
# inputcombo
input_combo_price_list = list(combo_issue_agent_obj.values_list('combo_issue_request__input_combo__name', 'combo_issue_request__input_combo__price', 'combo_issue_request__input_combo__inputpart__value','combo_issue_request__quantity_for_area'))
input_combo_price_df = pd.DataFrame(input_combo_price_list, columns=['input_combo_name', 'input_combo_price', 'input_combo_weight', 'quantity_of_area'])


In [5]:
input_combo_price_df

,input_combo_name,input_combo_price,input_combo_weight,quantity_of_area
0,One Acre Seed Pkt,10.00,100.000,10.00
1,One Acre Seed Pkt,10.00,100.000,20.00
2,One Acre Seed Pkt,10.00,100.000,15.00
3,One Acre Seed Pkt,10.00,100.000,9.00
4,One Acre Seed Pkt,10.00,100.000,6.00


In [11]:
price_change_list = list(AgentDistributionPriceChangeLog.objects.filter(agent__userclustermap__season_id=3).values_list('agent__first_name', 'agent__userprofile__code', 'from_date', 'to_date', 'price', 'agent__userclustermap__cluster__name'))
price_change_column = ['agent_name', 'agent_code', 'from_date', 'to_date', 'price', 'cluster_name']
price_change_df = pd.DataFrame(price_change_list, columns=price_change_column)
price_change_log_dict = price_change_df.groupby('agent_code').apply(lambda x: x.to_dict('r')).to_dict()

In [12]:
price_change_df

,agent_name,agent_code,from_date,to_date,price,cluster_name
0,Muruga_Terakanambi,AT068,2021-06-04,2021-06-04,8.800,Terakanambi
1,Mahesh K,AT048,2021-06-04,2021-06-04,8.800,Arakalavadi
2,Shivakumar,AT011,2021-06-04,2021-06-04,8.800,Arakalavadi
3,Muruga_Sargur,AY018,2021-06-04,2021-06-04,8.800,HD Kote
4,Mahadeva Prasad,AS062,2021-06-04,2021-06-04,9.100,HD Kote
5,Gopala H,AY009,2021-06-04,2021-06-04,9.100,Nanjangud


In [6]:
def predict_yield(crop_id, harvest_level_name, area):
    return YeildPrediction.objects.get(
        crop_id=crop_id, harvest_range__harvest_name=harvest_level_name
    ).expected_yeild_weight_in_kg * Decimal(area)

In [19]:
data = {
    'season_id': 3, 
    'filter': 'both', 
    'agent_ids': [],
    'cluster_ids': [1, 2, 3, 4, 6, 7, 8, 11], 
    'super_visor_ids': [], 
    'selected_date': '2021-06-07'
}

In [24]:
data = {
    'season_id': 3, 
    'filter': 'both', 
    'agent_ids': [],
    'cluster_ids': [], 
    'super_visor_ids': [], 
    'selected_date': '2021-06-07'
}

# cluster
if len(data["cluster_ids"]) == 0:
    cluster_ids = list(Cluster.objects.all().values_list("id", flat=True))
else:
    cluster_ids = list(Cluster.objects.filter(id__in=data["cluster_ids"]).values_list("id", flat=True))
# super_visor
if len(data["super_visor_ids"]) == 0:
    supervisor_ids = list(UserProfile.objects.filter(user_type_id=5).values_list("user_id", flat=True))
else:
    supervisor_ids = list(UserProfile.objects.filter(user_type_id=5, user_id__in=data["super_visor_ids"]).values_list("user_id", flat=True))
# agent
if len(data["agent_ids"]) == 0:
    agent_ids = list(UserProfile.objects.filter(user_type_id=6).values_list("user_id", flat=True))
else:
    agent_ids = list(UserProfile.objects.filter(user_type_id=6, user_id__in=data["agent_ids"]).values_list("user_id", flat=True))

#  farmer cluster
cluster_based_farmers_ids = list(FarmerClusterSeasonMap.objects.filter(season_id=data['season_id'], cluster_id__in=cluster_ids).values_list("farmer_id", flat=True))

# supervisor_based
user_farmer_ids = list(UserFarmerMap.objects.filter(officer_id__in=supervisor_ids,farmer__season_id=data['season_id'],
                                                    farmer__farmer__id__in=cluster_based_farmers_ids)
                       .values_list("farmer__farmer_id", flat=True))

# agent farmer map
farmer_ids = list(
    AgentFarmerMap.objects.filter(
        agent_id__in=agent_ids,
        farmer__season_id=data['season_id'],
        farmer__farmer__id__in=user_farmer_ids,
    ).values_list("farmer__farmer_id", flat=True)
)

# make sowing df
values = (Sowing.objects.filter(cultivation_phase_id=2, season_id=data['season_id'], farmer_id__in=farmer_ids, is_active=True)
        .values_list("id", "sowing_date").values_list("id", "sowing_date", "area", "crop_id"))
columns = ["id", "sowing_date", "area", "crop_id"]
df = pd.DataFrame(list(values), columns=columns)
forecast_date = datetime.datetime.strptime(data["selected_date"], "%Y-%m-%d").date()

df["forecast_date"] = forecast_date

# find sowing age baseed on forecast date
df["date_diff"] = (df["forecast_date"] - df["sowing_date"]).dt.days

# make dict for loop purpose {'harvest_Count': 'harvest_days'} eg: {'1': 55}
harvest_levels = dict(HarvestLevel.objects.filter().values_list("harvest_name", "harvest_interval_duration_in_days"))

# make df only harvest sowing for forecast date
df = df[df["date_diff"].isin(harvest_levels.values())]

# sort df by harvest days
df = df.sort_values(by=["date_diff"])

# create 'harvest_count' column
df.assign(**{"harvest_count": None})
data = {"transplanted_area_wise": {}, "transplanted_quantity_wise": {}}

# find and apply harvest count for forecast date
if not df.empty:
    for harvest_count in harvest_levels:
        df.loc[df["date_diff"] == int(harvest_levels[harvest_count]), "harvest_count"] = harvest_count

    # find yield for every sowing
    df["yield_in_kg"] = df.apply(lambda row: predict_yield(row["crop_id"], row["harvest_count"], row["area"]),axis=1,)

    for harvest_count in df["date_diff"].unique():
        data["transplanted_area_wise"][str(harvest_count)] = df[df["date_diff"] == harvest_count]["area"].sum()
        data["transplanted_quantity_wise"][str(harvest_count)] = round(df[df["date_diff"] == harvest_count]["yield_in_kg"].sum() / 1000, 2)

    data["total_transplanted_area"] = round(df["area"].sum(), 2)
    data["total_transplanted_qty"] = round(df["yield_in_kg"].sum() / 1000 , 2)

else:
    data = "empty"
data

{'transplanted_area_wise': {'55': 5.0, '62': 3.0, '69': 2.1, '76': 7.0},
 'transplanted_quantity_wise': {'55': Decimal('1.25'),
  '62': Decimal('1.80'),
  '69': Decimal('2.52'),
  '76': Decimal('10.50')},
 'total_transplanted_area': 17.1,
 'total_transplanted_qty': Decimal('16.07')}

In [7]:
season_id = 3
cluster_ids = []
agent_ids = []
supervisor_ids = []
from_date = '2021-06-1'
to_date = '2021-06-30'
# cluster
if len(cluster_ids) == 0:
    cluster_ids = list(Cluster.objects.all().values_list("id", flat=True))
else:
    cluster_ids = list(Cluster.objects.filter(id__in=cluster_ids).values_list("id", flat=True))
# super_visor
if len(supervisor_ids) == 0:
    supervisor_ids = list(UserProfile.objects.filter(user_type_id=5).values_list("user_id", flat=True))
else:
    supervisor_ids = list(UserProfile.objects.filter(user_type_id=5, user_id__in=supervisor_ids).values_list("user_id", flat=True))
# agent
if len(agent_ids) == 0:
    agent_ids = list(UserProfile.objects.filter(user_type_id=6).values_list("user_id", flat=True))
else:
    agent_ids = list(UserProfile.objects.filter(user_type_id=6, user_id__in=agent_ids).values_list("user_id", flat=True))

#  farmer cluster
cluster_based_farmers_ids = list(FarmerClusterSeasonMap.objects.filter(season_id=season_id, cluster_id__in=cluster_ids).values_list("farmer_id", flat=True))

# supervisor_based
user_farmer_ids = list(UserFarmerMap.objects.filter(officer_id__in=supervisor_ids,farmer__season_id=season_id,farmer__farmer__id__in=cluster_based_farmers_ids,).values_list("farmer__farmer_id", flat=True))

# agent farmer map
farmer_ids = list(AgentFarmerMap.objects.filter(agent_id__in=agent_ids,farmer__season_id=season_id,farmer__farmer__id__in=user_farmer_ids,).values_list("farmer__farmer_id", flat=True))

sowing_values = (Sowing.objects.filter(cultivation_phase_id=2, season_id=season_id, farmer_id__in=farmer_ids,is_active=True).values_list("id", "sowing_date").values_list("id", "sowing_date", "area", "crop_id"))
sowing_columns = ["id", "sowing_date", "area", "crop_id"]
sowing_df = pd.DataFrame(list(sowing_values), columns=sowing_columns)

date_from = datetime.datetime.strptime(from_date, "%Y-%m-%d")
date_to = datetime.datetime.strptime(to_date, "%Y-%m-%d")

date_range_difference = (date_to - date_from).days
forecast_dates = pd.date_range(date_from, periods=date_range_difference, freq="D")

harvest_levels = HarvestLevel.objects.filter()
harvest_interval = harvest_levels.values_list("harvest_interval_duration_in_days", flat=True)
harvest_key_pair = dict(harvest_levels.values_list("harvest_name", "harvest_interval_duration_in_days"))
data = {}

for forecast_date in forecast_dates:
    print(forecast_date)
    harvest_availabel_df = sowing_df.copy()
    harvest_availabel_df["forecast_date"] = forecast_date.date()
    harvest_availabel_df["date_diff"] = (harvest_availabel_df["forecast_date"] - harvest_availabel_df["sowing_date"]).dt.days
    harvest_availabel_df = harvest_availabel_df[harvest_availabel_df["date_diff"].isin(harvest_interval)]
    harvest_availabel_df = harvest_availabel_df.sort_values(by=["date_diff"])
    harvest_availabel_df.assign(**{"harvest_count": None})

    print(harvest_availabel_df)
    # assign harvest count for sowing
    for level in harvest_key_pair:
        print("for in level")
        harvest_availabel_df.loc[harvest_availabel_df["date_diff"] == int(harvest_key_pair[level]),"harvest_count",] = level

    harvest_availabel_df["yield_in_kg"] = harvest_availabel_df.apply(lambda row: predict_yield(row["crop_id"], row["harvest_count"], row["area"]),axis=1,)
    data[str(forecast_date.date())] = {
        "total_yield": (round(harvest_availabel_df["yield_in_kg"].sum(), 2) / 1000),
        "total_area": round(harvest_availabel_df["area"].sum()),
    }
    print("for done ")
    print(data)
    
df = pd.DataFrame(list(data.items()), columns=["Date", "Value"])
df = pd.DataFrame(data)
df_t = df.T

df_t

# # initializing excel
# writer = pd.ExcelWriter(str("static/media/") + "date_range_wise_forecast.xlsx", engine="xlsxwriter")
# # creating excel sheet with name

# df_t.to_excel(writer, sheet_name="Sheet1", startrow=1)

# # assigning that sheet to obj
# workbook = writer.book
# worksheet = writer.sheets["Sheet1"]
# merge_format = workbook.add_format(
#     {
#         "bold": 1,
#         "border": 1,
#         "align": "center",
#         "valign": "vcenter",
#         "fg_color": "yellow",
#     }
# )

# date = datetime.datetime.now().date()
# # Merge 3 cells.
# worksheet.merge_range("A1:C1","Date range forecast report for  " + str(date_from) + " " + str(date_to),merge_format,)

# format1 = workbook.add_format({"num_format": "#,##0.00"})

# # Set the column width and format.
# worksheet.set_column("B:B", 18, format1)
# worksheet.set_column(0, 7, 20)

# # Add a header format.
# header_format = workbook.add_format({"fg_color": "#D7E4BC"})

# # Write the column headers with the defined format.
# for col_num, value in enumerate(df.columns.values):
#     worksheet.write(0, col_num + 1, value, header_format)
# writer.save()
# document = {}
# try:
#     image_path = str("static/media/") + "date_range_wise_forecast.xlsx"
#     with open(image_path, "rb") as image_file:
#         encoded_image = b64encode(image_file.read())
#         document["excel"] = encoded_image
# except Exception as err:
#     print(err)
# document["data"] = data


2021-06-01 00:00:00
       id sowing_date  area  crop_id forecast_date  date_diff
40   6397  2021-04-07   1.0        1    2021-06-01         55
377  7298  2021-04-07   1.0        1    2021-06-01         55
385  8887  2021-03-31   0.5        1    2021-06-01         62
31   6385  2021-03-24   1.0        1    2021-06-01         69
for in level
for in level
for in level
for in level
for in level
for in level
for in level
for in level
for in level
for in level
for done 
{'2021-06-01': {'total_yield': Decimal('2.001'), 'total_area': 4.0}}
2021-06-02 00:00:00
       id sowing_date  area  crop_id forecast_date  date_diff
34   6392  2021-04-08   1.0        1    2021-06-02         55
375  7305  2021-04-08   0.5        1    2021-06-02         55
177  6574  2021-04-08   0.5        1    2021-06-02         55
137  6620  2021-04-08   1.0        1    2021-06-02         55
136  6633  2021-04-08   0.5        1    2021-06-02         55
133  6631  2021-04-08   0.5        1    2021-06-02         55
128  66

       id sowing_date  area  crop_id forecast_date  date_diff
3    8085  2021-04-15   1.0        1    2021-06-09         55
203  7443  2021-04-15   1.0        1    2021-06-09         55
205  6841  2021-04-15   1.0        1    2021-06-09         55
207  8049  2021-04-15   1.0        1    2021-06-09         55
249  6744  2021-04-15   1.0        1    2021-06-09         55
268  6849  2021-04-15   1.0        1    2021-06-09         55
270  6895  2021-04-15   0.5        1    2021-06-09         55
454  8947  2021-04-15   0.5        1    2021-06-09         55
271  6898  2021-04-15   1.0        1    2021-06-09         55
304  7354  2021-04-15   0.5        1    2021-06-09         55
405  7638  2021-04-15   1.0        1    2021-06-09         55
418  7680  2021-04-15   1.0        1    2021-06-09         55
419  7806  2021-04-15   2.0        1    2021-06-09         55
421  7828  2021-04-15   1.0        1    2021-06-09         55
422  7830  2021-04-15   0.5        1    2021-06-09         55
274  690

       id sowing_date  area  crop_id forecast_date  date_diff
127  7561  2021-04-20   0.5        1    2021-06-14         55
441  7985  2021-04-20   0.5        1    2021-06-14         55
416  7640  2021-04-20   0.5        1    2021-06-14         55
409  7653  2021-04-20   1.0        1    2021-06-14         55
392  7528  2021-04-20   0.5        1    2021-06-14         55
391  7446  2021-04-20   1.0        1    2021-06-14         55
298  8405  2021-04-20   0.5        1    2021-06-14         55
235  7096  2021-04-20   1.5        1    2021-06-14         55
215  7468  2021-04-20   1.0        1    2021-06-14         55
164  7114  2021-04-20   1.0        1    2021-06-14         55
158  8418  2021-04-20   1.5        1    2021-06-14         55
149  8414  2021-04-20   1.0        1    2021-06-14         55
146  7923  2021-04-20   1.0        1    2021-06-14         55
138  7569  2021-04-20   0.5        1    2021-06-14         55
131  7992  2021-04-20   0.5        1    2021-06-14         55
125  839

for done 
{'2021-06-01': {'total_yield': Decimal('2.001'), 'total_area': 4.0}, '2021-06-02': {'total_yield': Decimal('4.85'), 'total_area': 14.0}, '2021-06-03': {'total_yield': Decimal('3.30'), 'total_area': 9.0}, '2021-06-04': {'total_yield': Decimal('6.85'), 'total_area': 26.0}, '2021-06-05': {'total_yield': Decimal('3.675'), 'total_area': 10.0}, '2021-06-06': {'total_yield': Decimal('15.3775'), 'total_area': 23.0}, '2021-06-07': {'total_yield': Decimal('16.0721'), 'total_area': 17.0}, '2021-06-08': {'total_yield': Decimal('4.9255'), 'total_area': 10.0}, '2021-06-09': {'total_yield': Decimal('16.8035'), 'total_area': 38.0}, '2021-06-10': {'total_yield': Decimal('12.703'), 'total_area': 31.0}, '2021-06-11': {'total_yield': Decimal('19.576'), 'total_area': 40.0}, '2021-06-12': {'total_yield': Decimal('12.728'), 'total_area': 29.0}, '2021-06-13': {'total_yield': Decimal('24.0551'), 'total_area': 31.0}, '2021-06-14': {'total_yield': Decimal('23.535'), 'total_area': 39.0}, '2021-06-15': {

for done 
{'2021-06-01': {'total_yield': Decimal('2.001'), 'total_area': 4.0}, '2021-06-02': {'total_yield': Decimal('4.85'), 'total_area': 14.0}, '2021-06-03': {'total_yield': Decimal('3.30'), 'total_area': 9.0}, '2021-06-04': {'total_yield': Decimal('6.85'), 'total_area': 26.0}, '2021-06-05': {'total_yield': Decimal('3.675'), 'total_area': 10.0}, '2021-06-06': {'total_yield': Decimal('15.3775'), 'total_area': 23.0}, '2021-06-07': {'total_yield': Decimal('16.0721'), 'total_area': 17.0}, '2021-06-08': {'total_yield': Decimal('4.9255'), 'total_area': 10.0}, '2021-06-09': {'total_yield': Decimal('16.8035'), 'total_area': 38.0}, '2021-06-10': {'total_yield': Decimal('12.703'), 'total_area': 31.0}, '2021-06-11': {'total_yield': Decimal('19.576'), 'total_area': 40.0}, '2021-06-12': {'total_yield': Decimal('12.728'), 'total_area': 29.0}, '2021-06-13': {'total_yield': Decimal('24.0551'), 'total_area': 31.0}, '2021-06-14': {'total_yield': Decimal('23.535'), 'total_area': 39.0}, '2021-06-15': {

for done 
{'2021-06-01': {'total_yield': Decimal('2.001'), 'total_area': 4.0}, '2021-06-02': {'total_yield': Decimal('4.85'), 'total_area': 14.0}, '2021-06-03': {'total_yield': Decimal('3.30'), 'total_area': 9.0}, '2021-06-04': {'total_yield': Decimal('6.85'), 'total_area': 26.0}, '2021-06-05': {'total_yield': Decimal('3.675'), 'total_area': 10.0}, '2021-06-06': {'total_yield': Decimal('15.3775'), 'total_area': 23.0}, '2021-06-07': {'total_yield': Decimal('16.0721'), 'total_area': 17.0}, '2021-06-08': {'total_yield': Decimal('4.9255'), 'total_area': 10.0}, '2021-06-09': {'total_yield': Decimal('16.8035'), 'total_area': 38.0}, '2021-06-10': {'total_yield': Decimal('12.703'), 'total_area': 31.0}, '2021-06-11': {'total_yield': Decimal('19.576'), 'total_area': 40.0}, '2021-06-12': {'total_yield': Decimal('12.728'), 'total_area': 29.0}, '2021-06-13': {'total_yield': Decimal('24.0551'), 'total_area': 31.0}, '2021-06-14': {'total_yield': Decimal('23.535'), 'total_area': 39.0}, '2021-06-15': {

,total_yield,total_area
2021-06-01,2.001,4
2021-06-02,4.85,14
2021-06-03,3.30,9
2021-06-04,6.85,26
2021-06-05,3.675,10
2021-06-06,15.3775,23
2021-06-07,16.0721,17
2021-06-08,4.9255,10
2021-06-09,16.8035,38
2021-06-10,12.703,31


In [13]:
agent_inventry_obj = list(AgentInventory.objects.filter(season_id=3).values_list('agent_id', 'agent__first_name', 'combo_issue_request__input_combo_id', 'combo_issue_request__quantity_in_numbers', 'combo_issue_request__input_combo__name'))      
agent_inventry_column = ['agent_id', 'agent_name', 'input_combo_id', 'quantity',  'input_combo_name']
agent_inventry_df = pd.DataFrame(agent_inventry_obj, columns=agent_inventry_column)
agent_inventry_df = agent_inventry_df.groupby(['agent_id', 'input_combo_id']).agg({'agent_name': 'first', 'quantity': sum, 'input_combo_name': 'first'}).reset_index()
agent_inventry_df = agent_inventry_df.drop(columns=['input_combo_id']) 


#convert_groupby_table_product_cost_row_wise_value_into_column(using_pandas_pivot_table)
final_df = pd.pivot_table(agent_inventry_df, index=['agent_id','agent_name'] , columns='input_combo_name', aggfunc=min, fill_value=0)

#convert_pivot_table_to_normal_df
final_df.columns = final_df.columns.droplevel(0) #remove amount
final_df.columns.name = None  #remove categories
final_df = final_df.reset_index() #index to columns

final_df

,agent_id,agent_name,17:17:17,1st KIT,Half Acre Seed Pkt,One Acre Seed Pkt,Pesticide pack -3 dipping kit
0,31,Arasashetty,0,0,40,15,0
1,32,Bangaranaika,0,0,60,50,0
2,34,H N Papanna,0,0,80,20,0
3,35,Madappa,0,0,50,75,0
4,37,Mahadevaprasad B S,0,0,40,50,0
5,39,Manikanta,0,0,20,10,0
6,40,Shanidevara Murthy B K,0,0,50,70,0
7,41,Shivamahadevu,0,0,60,60,0
8,43,Gopala H,0,30,190,215,400
9,44,M Kumaraswamy,0,0,40,40,75


In [6]:
SowingBoundary.objects.filter(sowing__season_id=3).count()

1252

In [32]:
Sowing.objects.filter(season_id=3, cultivation_phase_id=2).count()

906

In [30]:
count = 0
for sowing in sowing_obj:
    if SowingBoundary.objects.filter(sowing_id=sowing.id).exists():
        count += 1  
        sowing_boundry_obj = SowingBoundary.objects.filter(sowing_id=sowing.id)
        sowing_boundry_map_obj = SowingBoundaryMap(
                                                    sowing=sowing, 
                                                    data_source_id=3, 
                                                    added_by_id=sowing.farmer.created_by_id
                                                    )
        
        sowing_boundry_map_obj.save()
        sowing_boundry_map_obj.sowing_boundry.add(*sowing_boundry_obj)
        sowing_boundry_map_obj.save()
print(count)

257


In [ ]:
sowing=, sowing_boundry=, data_source=, added_by=

In [3]:
farmer_id_list = list(AgentFarmerMap.objects.filter(agent_id=48, farmer__season_id=3).values_list('farmer__farmer_id', flat=True))

farmer_code_list = list(Sowing.objects.filter(farmer_id__in=farmer_id_list, cultivation_phase_id=2, season_id=3).values_list('farmer__code', flat=True))

list(FarmerClusterSeasonMap.objects.filter(farmer__code__in=farmer_code_list, season_id=3).values_list('seasonal_farmer_code', flat=True))

In [ ]:
cultivation_phase_df = cultivation_phase_df.drop_duplicates()
cultivation_phase_df = cultivation_phase_df.sort_values(by=['farmer_code'])
duplicate = cultivation_phase_df[cultivation_phase_df.duplicated('farmer_code')]
duplicate_list = list(duplicate['farmer_code'])
duplicate_df  = cultivation_phase_df[cultivation_phase_df['farmer_code'].isin(duplicate_list) ]

from IPython.core.display import HTML
display(HTML(duplicate_df.to_html()))

duplicate_df['farmer_code']

In [19]:
season_id = 3
cluster_id_list = []

if len(cluster_id_list) == 0:
    cluster_id_list = list(ClusterSeasonMap.objects.filter(season_id=season_id).values_list('cluster_id', flat=True))

# farmer agent cluster df
farmer_agent_list = list(FarmerClusterSeasonMap.objects.filter(season_id=season_id, cluster_id__in=cluster_id_list).values_list('farmer_id', 'farmer__first_name', 'farmer__last_name', 'seasonal_farmer_code', 'cluster__name', 'agentfarmermap__agent__first_name'))
farmer_agent_column = ['farmer_id', 'first_name', 'last_name', 'farmer_code', 'cluster_name', 'agent_name']
farmer_agent_df = pd.DataFrame(farmer_agent_list, columns=farmer_agent_column)
farmer_agent_df = farmer_agent_df.drop_duplicates('farmer_id')

# farmer sowing df
farmer_sowing_list = list(Sowing.objects.filter(season_id=season_id, farmer_id__in=list(farmer_agent_df['farmer_id'])).values_list('farmer_id', 'farmer__mobile', 'farmer__village__name', 'season__name', 'area_calculated_via_geo_fencing'))            
farmer_sowing_column = ['farmer_id', 'mobile', 'village', 'season_name', 'geo_calculated_area']
farmer_sowing_df = pd.DataFrame(farmer_sowing_list, columns=farmer_sowing_column)
farmer_sowing_df = farmer_sowing_df.fillna(0)
farmer_sowing_df_geo = farmer_sowing_df.groupby('farmer_id').agg({'mobile':'first', 'village':'first', 'season_name':'first','geo_calculated_area':sum}).reset_index()

combain_df = pd.merge(farmer_agent_df, farmer_sowing_df_geo, left_on='farmer_id', right_on='farmer_id', how='outer')
combain_df = combain_df.drop_duplicates('farmer_id')

# cultivation pase
phase_list = list(Sowing.objects.filter(season_id=season_id, farmer_id__in=list(combain_df['farmer_id'])).values_list('farmer_id', 'cultivation_phase_id', 'area', 'water_source__name'))
phase_column = ['farmer_id', 'pase_id', 'area', 'main_crop_water_resource_name']
phase_df = pd.DataFrame(phase_list, columns=phase_column)

phase1_df = phase_df[phase_df['pase_id']==1].drop(columns=['pase_id'])
phase1_df = phase1_df.rename(columns={'area': 'total_nursury_crop_area'})
phase1_df = phase1_df.groupby('farmer_id').agg({'total_nursury_crop_area': sum,'main_crop_water_resource_name': 'first'}).reset_index()

phase2_df = phase_df[phase_df['pase_id']==2].drop(columns=['pase_id', 'main_crop_water_resource_name'])
phase2_df = phase2_df.rename(columns={'area': 'total_transplant_crop_area'})
phase2_df = phase2_df.groupby('farmer_id').agg({'total_transplant_crop_area': sum}).reset_index()

cultivation_phase_df = pd.merge(phase1_df, phase2_df, left_on='farmer_id', right_on='farmer_id', how='outer')
cultivation_phase_df = cultivation_phase_df.fillna(0)

combain_df = pd.merge(combain_df, cultivation_phase_df, left_on='farmer_id', right_on='farmer_id', how='left')

# farmer bank df
farmer_bank_list = FarmerBankDetails.objects.filter(farmer_id__in=list(combain_df['farmer_id']), is_primary=True).values_list("farmer_id", "bank", "branch", "ifsc_code", "micr_code", "account_holder_name", "account_number")                
farmer_bank_column = ["farmer_id", "bank", "branch", "ifsc_code", "micr_code", "account_holder_name", "account_number"]
farmer_bank_df = pd.DataFrame(farmer_bank_list, columns=farmer_bank_column).fillna(0).drop_duplicates('farmer_id')
combain_df = pd.merge(combain_df, farmer_bank_df, left_on='farmer_id', right_on='farmer_id', how='left')

# farmer distribution sowing
sowing_list = list(Sowing.objects.filter(season_id=season_id, farmer_id__in=list(combain_df['farmer_id'])).values_list('id', flat=True))
if AgentFarmerDistributionSowing.objects.filter(sowing_id__in=sowing_list).exists():
    farmer_distribution_list = AgentFarmerDistributionSowing.objects.filter(sowing_id__in=sowing_list).values_list('sowing__farmer_id', 'input_combo__name', 'no_of_unit')
    farmer_distribution_column = ['farmer_id', 'input_combo', 'number_of_unit']
    farmer_distribution_df = pd.DataFrame(farmer_distribution_list, columns=farmer_distribution_column)
    farmer_distribution_df = farmer_distribution_df.groupby(['farmer_id', 'input_combo']).agg({'number_of_unit':sum}).reset_index()
    farmer_distribution_df

    #->##convert_groupby_table_product_cost_row_wise_value_into_column(using_pandas_pivot_table)
    farmer_distribution_df = pd.pivot_table(farmer_distribution_df, index=['farmer_id'] , columns='input_combo', aggfunc=min, fill_value=0)

    #->##convert_pivot_table_to_normal_df
    farmer_distribution_df.columns = farmer_distribution_df.columns.droplevel(0) #remove amount
    farmer_distribution_df.columns.name = None  #remove categories
    farmer_distribution_df = farmer_distribution_df.reset_index() #index to columns
    combain_df = pd.merge(combain_df, farmer_distribution_df, left_on='farmer_id', right_on='farmer_id', how='left').fillna(0)

# harvest 
sowing_id_list = list(Sowing.objects.filter(season_id=season_id, farmer_id__in=list(combain_df['farmer_id']), cultivation_phase_id=2).values_list('id', flat=True))
harvest_list = list(Harvest.objects.filter(sowing_id__in=sowing_id_list).values_list('sowing__farmer_id', 'value'))
harvest_column = ['farmer_id', 'harvest_value']
haevesr_df = pd.DataFrame(harvest_list, columns=harvest_column)
haevesr_df = haevesr_df.groupby('farmer_id').agg({'harvest_value':sum}).reset_index()

combain_df = pd.merge(combain_df, haevesr_df, left_on='farmer_id', right_on='farmer_id', how='left').fillna(0)
list(set(combain_df['cluster_name']))

['HD Kote',
 'Sargur',
 'Terakanambi',
 'Nanjangud',
 'gopalpura',
 'kodasoge',
 'Baragi',
 'Arakalavadi']

In [18]:
df = combain_df[combain_df['cluster_name'] =='gopalpura']
df['total_transplant_crop_area'].sum()
# df['total_nursury_crop_area'].sum()

656.5

In [ ]:
combain_df['Terakanambi']

In [4]:
sowing_df = pd.DataFrame(list(Sowing.objects.filter(season_id=season_id, farmer_id__in=list(combain_df['farmer_id'])).values('id', 'farmer_id')))
sowing_df[sowing_df['farmer_id']==6573]

,id,farmer_id
1181,6736,6573
1182,7495,6573


,farmer_id,first_name,last_name,farmer_code,cluster_name,agent_name,mobile,village,season_name,geo_calculated_area,total_nursury_crop_area,main_crop_water_resource_name,total_transplant_crop_area,bank,branch,ifsc_code,micr_code,account_holder_name,account_number
500,6573,Bolaregowda,Late Kempegowda,21HSF003,HD Kote,Praveen Kumar M M,9901226128,Gundathur,2021-Marigold-01,1.165,1.0,Borewell,1.0,Kaveri grameena bank,N belthur,PKGB0012391,0,Bolaregowda,12391100001953
503,6572,Praveenkumar M M,B H Mahadevappa,21HSF002,HD Kote,Praveen Kumar M M,8151835533,Malali,2021-Marigold-01,1.038,1.0,Borewell,1.0,Karnataka grameena bank,N. belthur,PKGB0012391,0,Praveen Kumar M M,12391100001235
504,6574,Ganesha G S,Shivannegowda,21HSF004,HD Kote,Praveen Kumar M M,9900696908,Gundathur,2021-Marigold-01,1.033,1.5,Borewell,1.5,Kaveri grameena bank,N belthur,PKGB0012391,0,Ganesha G S,12391100000186
505,6503,Naveenkumar M M,Mahadevappa B H,21HSF001,HD Kote,Praveen Kumar M M,9353820293,Malali,2021-Marigold-01,1.247,1.0,Borewell,1.0,Kaveri grameena bank,N belthur,PKGB0012391,0,Naveen Kumar M M,12391100005194
506,6576,Manikanta,Anandakumar B S,21HSF006,HD Kote,Praveen Kumar M M,9901271930,N. Belathuru,2021-Marigold-01,1.609,1.0,Borewell,1.0,Kaveri grameena bank,N belathur,PKGB0012391,0,Manikanta,12391100003556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4661,10650,Rajamma,Mahadevappa H B,21HSF068,HD Kote,Praveen Kumar M M,9964628201,Malali,2021-Marigold-01,0.000,1.0,Borewell,0.5,Bank of baroda,Heggadadevana kote,BARB0VJHEGG,0,Rajamma,83930100008005
4793,9668,Gopalaiah,Somanna,21HSF017,HD Kote,Praveen Kumar M M,8581954550,Sogahalli,2021-Marigold-01,0.000,0.5,Rain,0.5,Kaveri Grameena bank,Antarasanthe,PKGB0012007,0,Gopalaiah,12007100011447
4845,9684,Ashok M,Mahadevappa,21HSF033,HD Kote,Praveen Kumar M M,9902405572,Malali,2021-Marigold-01,2.070,2.0,Borewell,2.0,Karnataka bank ltd,Kanakadasanagar Mysuru,KARB0000623,0,Ashok M,6232500100547801
4861,9799,Thimmaiah,Gopaiah,21HSF052,HD Kote,Praveen Kumar M M,9535242367,Sogahalli,2021-Marigold-01,0.000,1.0,Rain,1.0,Kaveri Grameena Bank,Antharasanthe,PKGB0012007,0,Thimmaiah,12007100004636


In [26]:
import datetime

from decimal import * 

modified_by_id=99


procurement_obj = Procurement.objects.filter(procurement_group__season_id=3)

for item in procurement_obj:

    # print(item.procurement_group.payment_to_agent)

    if not AgentWallet.objects.filter(agent_id=item.procurement_group.agent_id).exists():

        updated_amount = item.procurement_group.payment_to_agent

        old_balance = 0

        print("type : ", updated_amount)

        AgentWallet.objects.create(

            agent_id=item.procurement_group.agent_id,

            current_balance=updated_amount,

            modified_by_id=modified_by_id,

        )

        new_balance = updated_amount

    else:

        old_balance = AgentWallet.objects.get(agent_id=item.procurement_group.agent_id).current_balance

        updated_amount = (old_balance + item.procurement_group.payment_to_agent)/8
        
        print(updated_amount/2)

        AgentWallet.objects.filter(agent_id=item.procurement_group.agent_id).update(current_balance=round(Decimal(updated_amount), 2), modified_by_id=modified_by_id)

        new_balance = updated_amount

    agent_transaction_log = AgentTransactionLog(

        date=datetime.datetime.today(),

        transaction_direction_id=3,  

        agent_id=item.procurement_group.agent_id,

        data_entered_by_id=modified_by_id,

        amount= item.procurement_group.payment_to_agent, #price need update

        transaction_id="1234",

        transaction_mode_id=1,

        transaction_approval_status_id=1,

        wallet_balance_before_this_transaction=old_balance,

        wallet_balance_after_this_transaction=new_balance,

        description="it will be given to agent hand after the wallet adjustment",

        modified_by_id=modified_by_id,

    )

    agent_transaction_log.save()


    input_procurement_transaction_log_obj = InputProcurementTransactionLog(

                                            transaction_log_id = agent_transaction_log.id,

                                            procurement_id = item.id)

    input_procurement_transaction_log_obj.save()


    agent_transaction_log = AgentTransactionLog(

        date=datetime.datetime.today(),

        transaction_direction_id=3,  

        agent_id=item.procurement_group.agent_id,

        data_entered_by_id=modified_by_id,

        amount= item.procurement_group.payment_to_wallet, #price need update

        transaction_id="1234",

        transaction_mode_id=1,

        transaction_approval_status_id=1,

        wallet_balance_before_this_transaction=old_balance,

        wallet_balance_after_this_transaction=new_balance,

        description="it will be given to agent hand after the wallet adjustment",

        modified_by_id=modified_by_id,

    )

    agent_transaction_log.save()

    input_procurement_transaction_log_obj = InputProcurementTransactionLog(

                                            transaction_log_id = agent_transaction_log.id,

                                            procurement_id = item.id)

    input_procurement_transaction_log_obj.save()

#     print('2')


3927.95
2410.151875
1476.6375
1771.3925
2042.56875
1787.62125
1548.9525
1683.01875
1162.9775
1705.86125
5414.23875
1152.8075
7758.418125
1579.0575
468.67625
1193.834375
1151.479375
2786.07
2218.155
664.869375
882.085
1224.55125
677.410625
5048.528125
2296.39125
1388.40875
1625.05875
1696.40125
798.20
1039.42625
407.65625
1798.198125
1094.178125
1594.60
2915.535625
617.11875
4097.79125
1477.631875
2430.051875
1515.00625
8447.66
1381.60
1495.150625
411.3225
379.35375
4678.799375
1186.665
1142.59375
4512.471875
4431.83375
5748.725
4250.92625
2109.790625
1947.59875
3979.208125
1727.17625
2100.550625
2159.955
1088.506875
877.925
12698.21375
5078.0575
432.838125
3320.165625
1713.9575
2095.09875
783.350625
1781.170625
3501.1375
1075.646875
281.335
1881.820625
1108.499375
4970.013125
1975.001875
182.49375
924.2125
1650.5275
799.541875
1235.1925
1109.155625
1289.72625
4777.245
369.611875
1094.344375
1439.440625
1391.655
1215.518125
2031.6775
1783.930625
179.79625
2557.57375
837.056875
900.25687

In [27]:

from decimal import *


agent_inventry_obj = AgentInventory.objects.filter(season_id = 3)


for item in agent_inventry_obj:

    print(item.agent_id)

    if not AgentWallet.objects.filter(agent_id=item.agent_id).exists():

        updated_amount = round(item.quantity_at_receipt * item.price_per_item)

        old_balance = 0

        print("type : ", updated_amount)

        AgentWallet.objects.create(

            agent_id=item.agent_id,

            current_balance=updated_amount,

            modified_by_id=99,

        )

        new_balance = updated_amount


    else:

        old_balance = AgentWallet.objects.get(agent_id=item.agent_id).current_balance

        updated_amount = old_balance + round(item.quantity_at_receipt * item.price_per_item)

        AgentWallet.objects.filter(agent_id=item.agent_id).update(current_balance=Decimal(updated_amount), modified_by=99)

        new_balance = updated_amount


    agent_transaction_log = AgentTransactionLog(

        date='2021-07-02',

        transaction_direction_id=2,  # ccgb to agent wallet when wallet adjustment - positive

        agent_id=item.agent_id,

        data_entered_by_id=99,

        amount= round(item.quantity_at_receipt * item.price_per_item),

        transaction_id="1234",

        transaction_mode_id=1,

        transaction_approval_status_id=1,

        wallet_balance_before_this_transaction=old_balance,

        wallet_balance_after_this_transaction=new_balance,

        description="it will be added as a positive amount in agent wallet as a commision for wallet adjustment",

        modified_by_id=99,

    )

    agent_transaction_log.save()


    input_distribution_transaction_map_obj = InputDistributionTransactionMap(

                                            agent_inventory_id = item.id,

                                            transaction_log_id = agent_transaction_log.id)

    input_distribution_transaction_map_obj.save()

    

        

    print(1)


90
1
122
1
48
1
43
1
91
1
58
1
121
1
74
1
59
1
120
1
46
1
77
1
44
1
138
1
50
1
55
1
91
1
58
1
134
1
139
1
141
1
65
1
135
1
145
1
122
1
90
1
48
1
87
1
85
1
137
1
140
1
110
1
43
1
90
1
48
1
43
1
91
1
90
1
122
1
48
1
43
1
63
1
77
1
121
1
71
1
72
1
73
1
143
1
56
1
73
1
103
1
120
1
60
1
121
1
56
1
73
1
51
1
56
1
50
1
74
1
120
1
59
1
146
1
50
1
122
1
63
1
142
1
91
1
139
1
141
1
134
1
145
1
65
1
43
1
135
1
58
1
71
1
72
1
60
1
49
1
121
1
59
1
143
1
51
1
103
1
146
1
138
1
50
1
137
1
144
type :  250
1
56
1
49
1
74
1
73
1
120
1
40
1
55
1
43
1
91
1
134
1
145
1
90
1
35
1
110
1
121
1
48
1
140
1
63
1
122
1
43
1
59
1
89
1
47
1
71
1
85
1
87
1
143
1
49
1
44
1
65
1
139
1
90
1
135
1
73
1
46
1
43
1
37
1
77
1
117
1
88
1
41
1
43
1
135
1
58
1
48
1
74
1
144
1
43
1
59
1
90
1
34
1
111
1
59
1
31
1
59
1
72
1
32
1
40
1
41
1
39
1
90
1
135
1
65
1
46
1
140
1
152
1
122
1
63
1
91
1
139
1
141
1
134
1
145
1
65
1
43
1
135
1
58
1
71
1
72
1
49
1
121
1
59
1
143
1
51
1
103
1
146
1
138
1
50
1
137
1
144
1
56
1
74
1
73
1
120
1
40

In [45]:


agent_id = 87


combo_return_request_obj = ComboReturnRequest.objects.filter(season_id=3, agent_id=agent_id)

for item in combo_return_request_obj:

    print(item.agent.id)


    if ComboReturnRequest.objects.filter(agent_id=item.agent_id, max_status_id=2, season_id=3).exists():

        print(item.agent_id)



        combo_return_request_ids = list(ComboReturnRequest.objects.filter(agent_id=item.agent_id, max_status_id=2, season_id=3).values_list('id', flat=True))



        for combo_return_request_id in combo_return_request_ids:

            combo_return_request_label_map_obj_count = ComboReturnRequestLabelMap.objects.filter(return_label__status_id=2, combo_return_request_id=combo_return_request_id).count()

            combo_return_request_label_map_obj = ComboReturnRequestLabelMap.objects.filter(return_label__status_id=2, combo_return_request_id=combo_return_request_id)


            if not AgentWallet.objects.filter(agent_id=item.agent_id).exists():

                updated_amount = round(combo_return_request_label_map_obj_count * combo_return_request_label_map_obj.return_label.agent_inventory.price_per_item) #price need update

                old_balance = 0

                print("type : ", updated_amount)

                AgentWallet.objects.create(

                    agent_id=item.agent_id,

                    current_balance=updated_amount,

                    modified_by_id=modified_by_id,

                )

                new_balance = updated_amount


            else:

                old_balance = AgentWallet.objects.get(agent_id=item.agent_id).current_balance

                updated_amount = old_balance + round(combo_return_request_label_map_obj_count * combo_return_request_label_map_obj.return_label.agent_inventory.price_per_item) #price need update

                AgentWallet.objects.filter(agent_id=item.agent_id).update(current_balance=Decimal(updated_amount), modified_by=request.user.id)

                new_balance = updated_amount


            agent_transaction_log = AgentTransactionLog(

                date=datetime.datetime.today(),

                transaction_direction_id=4,  # agent seed return

                agent_id=agent_id,

                data_entered_by_id=modified_by_id,

                amount= round(combo_return_request_label_map_obj_count * combo_return_request_label_map_obj.return_label.agent_inventory.price_per_item), #price need update

                transaction_id="1234",

                transaction_mode_id=1,

                transaction_approval_status_id=1,

                wallet_balance_before_this_transaction=old_balance,

                wallet_balance_after_this_transaction=new_balance,

                description="money adjustment to wallet - adding money to wallet(positive)",

                modified_by_id=modified_by_id,

            )

            agent_transaction_log.save()



            input_return_transaction_log_obj = InputReturnTransactionLog(

                                                transaction_log_id = agent_transaction_log.id,

                                                combo_return_request_id = combo_return_request_id

                                                )

            input_return_transaction_log_obj.save()

    else:

        print(0)



87
87


AttributeError: 'QuerySet' object has no attribute 'return_label'

In [25]:
InputDistributionTransactionMap.objects.all().delete()
InputReturnTransactionLog.objects.all().delete()
InputProcurementTransactionLog.objects.all().delete()

(0, {'inputdistri.InputProcurementTransactionLog': 0})

In [10]:
round(Decimal(2/3), 2)

Decimal('0.67')

In [32]:
date_list = list(set(list(InputReturnTransactionLog.objects.filter(transaction_log__agent_id=request.data['agent_id']).values_list('transaction_log__date', flat=True))))

In [80]:
input_retutn_list = InputReturnTransactionLog.objects.filter().values_list('combo_return_request_id', 'combo_return_request__input_combo__name', 'combo_return_request__comboreturnrequestlabelmap__return_label__label', 'combo_return_request__comboreturnrequestlabelmap__return_label__label', 'transaction_log__wallet_balance_before_this_transaction', 'transaction_log__wallet_balance_after_this_transaction', 'transaction_log__amount')
input_retutn_column = ['return_request_id', 'input_combo', 'quantity', 'label', 'wallet_before', 'wallet_after', 'amount']
input_retutn_df = pd.DataFrame(input_retutn_list, columns=input_retutn_column)
input_retutn_df = input_retutn_df.groupby('return_request_id').agg({'input_combo': 'first', 'wallet_before':'first', 'wallet_after': 'first', 'amount': 'first', 'label':list, 'quantity':'count'}).reset_index()
input_retutn_df['label'] = input_retutn_df['label'].astype(str)
input_retutn_df.to_dict('r')

[{'return_request_id': 15,
  'input_combo': '1st KIT',
  'wallet_before': Decimal('27351.00'),
  'wallet_after': Decimal('27351.00'),
  'amount': Decimal('23311.50'),
  'label': "['CCGB200', 'CCGB201']",
  'quantity': 2}]

In [81]:
sub_store_list = list(SubStoreIssueLabelAgentMap.objects.filter(agent_id=48).values_list('agent_inventory__combo_issue_request__input_combo__name', 'agent_inventory__combo_issue_request__input_combo_id'))
sub_store_column = ['combo_name', 'combo_id']
sub_store_df = pd.DataFrame(sub_store_list, columns=sub_store_column)
sub_store_df = sub_store_df.drop_duplicates()
master_dict = sub_store_df.to_dict('r')
master_dict

[{'combo_name': 'Pesticide pack -3 dipping kit', 'combo_id': 14},
 {'combo_name': '1st KIT', 'combo_id': 3},
 {'combo_name': 'Half Acre Seed Pkt', 'combo_id': 9},
 {'combo_name': 'One Acre Seed Pkt', 'combo_id': 8}]

In [3]:
agent_id = 48
combo_id = 14
    sub_store_list = list(SubStoreIssueLabelAgentMap.objects.filter(agent_id=agent_id, agent_inventory__combo_issue_request__input_combo_id=combo_id).values_list('agent_id', 'label', 'label', 'agent_inventory__combo_issue_request__input_combo__area__quantity_in_acre'))
    sub_store_column = ['agent_id', 'lable', 'lable_count', 'area']
    sub_store_df = pd.DataFrame(sub_store_list, columns=sub_store_column)
    sub_store_df = sub_store_df.drop_duplicates()
    sub_store_df = sub_store_df.groupby('agent_id').agg({'lable':list, 'lable_count':'count', 'area':sum}).reset_index()
    master_dict = sub_store_df.to_dict('r')
    master_dict
# sub_store_df

IndentationError: unexpected indent (<ipython-input-3-285277d26593>, line 3)

In [9]:
farmer_list = list(AgentFarmerMap.objects.filter(agent_id=103, farmer__season_id=3, farmer__cluster_id=11).values_list('farmer__farmer_id', flat=True))

In [15]:
Sowing.objects.filter(season_id=3, farmer_id__in=farmer_list, cultivation_phase_id=2, is_active=True).count()

9

In [37]:
2021072001124

In [42]:
df1 = pd.read_excel('QueryReport20210722(1).xls')
df2 = pd.DataFrame(list(TempProcurement.objects.filter(is_uploaded=False).values()))

df1 = df1.groupby('Ticket No').agg({'Gross time': 'first', 'Tare Time': 'first'}).reset_index().rename(columns={'Ticket No': 'ticket_number'})
df2 = df2.groupby('ticket_number').agg({'gross_time': 'first', 'tare_time': 'first'}).reset_index()

df = pd.merge(df1,df2, left_on='ticket_number', right_on='ticket_number', how='left')
df[['gross_time', 'tare_time', 'Gross time', 'Tare Time']] = df[['gross_time', 'tare_time', 'Gross time', 'Tare Time']].astype(str)

df['gross_time'] = df['gross_time'].str[:19]
df['tare_time'] = df['tare_time'].str[:19]

# grosstime_diff = df[df['gross_time'] != df['Gross time']]
# taretime_diff = df[df['tare_time'] != df['Tare Time']]
# grosstime_diff

df_tick=df[df['ticket_number']=='2021072001124']
df_tick

,ticket_number,Gross time,Tare Time,gross_time,tare_time
16,2021072001124,2021-07-20 23:16:21,2021-07-21 04:41:54,2021-07-20 23:16:21,2021-07-21 04:41:54


In [3]:
def get_active_season_id():
    season_obj = Season.objects.get(is_active=True)
    return season_obj.id

In [57]:
season_id = get_active_season_id()

available_cluster_ids = list(UserClusterMap.objects.filter(user_id=23, season_id=season_id).values_list("cluster_id", flat=True))
available_cluster = Cluster.objects.filter(id__in=available_cluster_ids).order_by("id")
farmer_cluster_list = FarmerClusterSeasonMap.objects.filter(cluster_id__in=available_cluster_ids, season_id=season_id).order_by("id").values_list(
    'farmer_id', 'farmer__first_name', 'farmer__last_name', 'seasonal_farmer_code', 'farmer__village__name', 'farmer__mobile', 'agentfarmermap__agent__first_name')
farmer_cluster_column = ['Farmer Id', 'First Name', 'Last Name', 'Farmer Code', 'Village', 'Phone', 'Agent Name']
farmer_cluster_df = pd.DataFrame(farmer_cluster_list, columns=farmer_cluster_column)


sowing_obj = Sowing.objects.filter(farmer_id__in=farmer_cluster_df['Farmer Id'], season_id=season_id)

#farmers without cp1 df
##cultivation phase 1 having farmers ids
farmers_with_cp1_id_list = list(sowing_obj.filter(cultivation_phase_id=1).values_list('farmer_id', flat=True))
##cultivation phase 1 not having farmers ids
farmers_without_cp1_id_list = list(set(farmer_cluster_df['Farmer Id'])-set(farmers_with_cp1_id_list))
##farmers without cp1 df
farmers_without_cp1_df = farmer_cluster_df[farmer_cluster_df['Farmer Id'].isin(farmers_without_cp1_id_list)] 

#farmers without cp2 df
##cultivation phase 2 having farmers ids
farmers_with_cp2_id_list = list(sowing_obj.filter(cultivation_phase_id=2).values_list('farmer_id', flat=True))
##cultivation phase 2 not having farmers ids
farmers_without_cp2_id_list = list(set(farmer_cluster_df['Farmer Id'])-set(farmers_with_cp2_id_list))
##farmers without cp2 df
farmers_without_cp2_df = farmer_cluster_df[farmer_cluster_df['Farmer Id'].isin(farmers_without_cp2_id_list)] 

#farmers without cp1 geo fence df
##farmers without cp1 geo fence list
farmers_without_cp1_fence_id_list = list(sowing_obj.filter(cultivation_phase_id=1, area_calculated_via_geo_fencing__isnull=True).values_list('farmer_id', flat=True))
#farmers without cp1 geo fence df
farmers_without_cp1_fence_df = farmer_cluster_df[farmer_cluster_df['Farmer Id'].isin(farmers_without_cp1_fence_id_list)] 

#farmers without cp2 geo fence df
##farmers without cp2 geo fence list
farmers_without_cp2_fence_id_list = list(sowing_obj.filter(cultivation_phase_id=2, area_calculated_via_geo_fencing__isnull=True).values_list('farmer_id', flat=True))
#farmers without cp2 geo fence df
farmers_without_cp2_fence_df = farmer_cluster_df[farmer_cluster_df['Farmer Id'].isin(farmers_without_cp2_fence_id_list)] 

#farmers without cp2 harvest df
##farmers with cp2 harvest
farmers_with_cp2_harvest_id_list = list(sowing_obj.filter(cultivation_phase_id=2).values_list('harvest__sowing__farmer_id', flat=True))
##farmer without cp2 harvest
farmers_without_cp2_harvest_id_list = list(set(farmer_cluster_df['Farmer Id'])-set(farmers_with_cp2_harvest_id_list))
##farmer without cp2 harvest df
farmers_without_cp2_harvest_df= farmer_cluster_df[farmer_cluster_df['Farmer Id'].isin(farmers_without_cp2_harvest_id_list)] 

#farmers without bank details df
##farmer with bank details
farmers_with_bank_details_list = list(FarmerBankDetails.objects.filter(farmer_id__in=farmer_cluster_df['Farmer Id']).values_list('farmer_id', flat=True))
##farmers without bank details
farmers_without_bank_details_list = list(set(farmer_cluster_df['Farmer Id'])-set(farmers_with_bank_details_list))
##farmers without bank details df
farmers_without_bank_details_df = farmer_cluster_df[farmer_cluster_df['Farmer Id'].isin(farmers_without_bank_details_list)] 


writer = pd.ExcelWriter(str("static/media/") + "master_farmers_report.xlsx", engine="xlsxwriter")

df_dict = {
    "Farmers And Details": farmer_cluster_df.drop(['Farmer Id'], axis=1),
    "Farmers Without Nursery Sowing": farmers_without_cp1_df.drop(['Farmer Id'], axis=1),
    "Farmers Without Transplanted Sowing": farmers_without_cp2_df.drop(['Farmer Id'], axis=1),
    "Farmers Having Nursery Sowing But Cultivation Area Not Calculated": farmers_without_cp1_fence_df.drop(['Farmer Id'], axis=1),
    "Farmers Having Transplanted Sowing But Cultivation Area Not Calculated": farmers_without_cp2_fence_df.drop(['Farmer Id'], axis=1),
    "Farmers Having Transplanted Sowing But Not Having Harvest": farmers_without_cp2_harvest_df.drop(['Farmer Id'], axis=1),
    "Farmers Without Bank Account Details": farmers_without_bank_details_df.drop(['Farmer Id'], axis=1)
}

for index, title in enumerate(df_dict):
    
    print(title)
    # creating excel sheet with name
    df_dict[title].to_excel(writer, sheet_name="sheet"+str(index), startrow=1, index=False)
    # assigning that sheet to obj
    workbook = writer.book
    worksheet = writer.sheets["sheet"+str(index)]
    merge_format = workbook.add_format(
        {
            "bold": 1,
            "border": 1,
            "align": "center",
            "valign": "vcenter",
            "fg_color": "yellow",
        }
    )

    date = datetime.datetime.now().date()
    # Merge 3 cells.
    worksheet.merge_range("A1:G1", title + " - " + str(date), merge_format)

    format1 = workbook.add_format({"num_format": "#,##0.00"})

    # Set the column width and format.
    worksheet.set_column("B:B", 18, format1)
    worksheet.set_column(0, 17, 20)

    # Add a header format.
    header_format = workbook.add_format({"fg_color": "#D7E4BC"})

    # Write the column headers with the defined format.
    for col_num, value in enumerate(df_dict[title].columns.values):
        worksheet.write(0, col_num + 1, value, header_format)
writer.save()



In [69]:
writer = pd.ExcelWriter(str("static/media/") + "master_farmers_report.xlsx", engine="xlsxwriter")

df_dict = {
    "Farmers And Details": farmer_cluster_df.drop(['Farmer Id'], axis=1),
    "Farmers Without Nursery Sowing": farmers_without_cp1_df.drop(['Farmer Id'], axis=1),
    "Farmers Without Transplanted Sowing": farmers_without_cp2_df.drop(['Farmer Id'], axis=1),
    "Farmers Having Nursery Sowing But Cultivation Area Not Calculated": farmers_without_cp1_fence_df.drop(['Farmer Id'], axis=1),
    "Farmers Having Transplanted Sowing But Cultivation Area Not Calculated": farmers_without_cp2_fence_df.drop(['Farmer Id'], axis=1),
    "Farmers Having Transplanted Sowing But Not Having Harvest": farmers_without_cp2_harvest_df.drop(['Farmer Id'], axis=1),
    "Farmers Without Bank Account Details": farmers_without_bank_details_df.drop(['Farmer Id'], axis=1)
}

for index, title in enumerate(df_dict):
    
    print(title)
    # creating excel sheet with name
    df_dict[title].to_excel(writer, sheet_name="sheet"+str(index), startrow=1, index=False)
    # assigning that sheet to obj
    workbook = writer.book
    worksheet = writer.sheets["sheet"+str(index)]
    merge_format = workbook.add_format(
        {
            "bold": 1,
            "border": 1,
            "align": "center",
            "valign": "vcenter",
            "fg_color": "yellow",
        }
    )

    date = datetime.datetime.now().date()
    # Merge 3 cells.
    worksheet.merge_range("A1:G1", title + " - " + str(date), merge_format)

    format1 = workbook.add_format({"num_format": "#,##0.00"})

    # Set the column width and format.
    worksheet.set_column("B:B", 18, format1)
    worksheet.set_column(0, 17, 20)

    # Add a header format.
    header_format = workbook.add_format({"fg_color": "#D7E4BC"})

    # Write the column headers with the defined format.
    for col_num, value in enumerate(df_dict[title].columns.values):
        worksheet.write(0, col_num + 1, value, header_format)
writer.save()



Farmers And Details
Farmers Without Nursery Sowing
Farmers Without Transplanted Sowing
Farmers Having Nursery Sowing But Cultivation Area Not Calculated
Farmers Having Transplanted Sowing But Cultivation Area Not Calculated
Farmers Having Transplanted Sowing But Not Having Harvest
Farmers Without Bank Account Details


In [49]:
season_id=3
print(season_id)
data = {'is_value_avaliable': True}

combo_issue_request_agentmap_obj = list(ComboIssueRequestAgentMap.objects.filter(combo_issue_request__season_id=season_id, combo_issue_request__max_status_id__in=[8,9], shop_id=1).values_list('agent_id', 'agent__first_name', 'combo_issue_request__input_combo_id', 'combo_issue_request__quantity_in_numbers','combo_issue_request__quantity_for_area', 'combo_issue_request__input_combo__name'))
combo_issue_request_agentmap_column = ['agent_id', 'agent_name', 'input_combo_id', 'quantity',  'area','input_combo_name']
combo_issue_request_agentmap_df = pd.DataFrame(combo_issue_request_agentmap_obj, columns=combo_issue_request_agentmap_column)

combo_issue_request_othershop_obj = list(ComboIssueRequestAgentMap.objects.filter(combo_issue_request__season_id=season_id, combo_issue_request__max_status_id=6).exclude(shop_id=1).values_list('agent_id', 'agent__first_name', 'combo_issue_request__input_combo_id', 'combo_issue_request__quantity_in_numbers', 'combo_issue_request__input_combo__name'))
column_names = ['agent_id', 'agent_name', 'input_combo_id', 'quantity',  'input_combo_name']
combo_issue_request_othershop_obj_df = pd.DataFrame(combo_issue_request_othershop_obj, columns=column_names)

merge_df = pd.concat([combo_issue_request_agentmap_df, combo_issue_request_othershop_obj_df])
merge_df1 = merge_df.groupby(['agent_id', 'input_combo_id']).agg({'agent_name': 'first', 'quantity': sum, 'input_combo_name': 'first'}).reset_index()
# merge_df1 = merge_df1.drop(columns=['input_combo_id']) 

input_combo_obj = InputCombo.objects.filter().values_list('id', 'name')
input_combo_df = pd.DataFrame(list(input_combo_obj), columns=['input_combo_id', 'name'])

inupt_merge_df = pd.merge(merge_df1, input_combo_df,  left_on='input_combo_id', right_on='input_combo_id', how='outer')
# inupt_merge_df.fillna(0)
merge_df1 = inupt_merge_df.drop(columns=['input_combo_id', 'input_combo_name']) 

# convert_groupby_table_product_cost_row_wise_value_into_column(using_pandas_pivot_table)
final_df = pd.pivot_table(merge_df1, index=['agent_id','agent_name'] , columns='name', aggfunc=min, fill_value=0)

#convert_pivot_table_to_normal_df
final_df.columns = final_df.columns.droplevel(0) #remove amount
final_df.columns.name = None  #remove categories

final_df = final_df.reset_index() #index to columns
# final_df1 = final_df.drop(columns=['agent_id'])

agent_farmer_obj = AgentFarmerMap.objects.filter(farmer__season_id=season_id)
agent_farmer_values = list(agent_farmer_obj.values_list('agent_id', 'agent__first_name', 'farmer__farmer__id', 'farmer__seasonal_farmer_code','farmer__farmer__first_name' ))
agent_farmer_columns = ["agent_id", 'agent_name', 'farmer_id', 'farmer_code', 'farmer_name']
agent_farmer_df = pd.DataFrame(agent_farmer_values, columns=agent_farmer_columns)

sowing_farmers_obj = Sowing.objects.filter(season_id=season_id)
sowing_farmers_values = list(sowing_farmers_obj.values_list('id','farmer_id', 'cultivation_phase_id', 'cultivation_phase__name', 'area', 'area_calculated_via_geo_fencing'))
sowing_farmers_columns = ["sowing_id","farmer_id", "cultivation_phase_id", "cultivation_phase_name", "area", "geo_fence"]
sowing_farmers_df = pd.DataFrame(sowing_farmers_values, columns=sowing_farmers_columns)

sowing_agent_farmer = pd.merge(agent_farmer_df, sowing_farmers_df, left_on='farmer_id', right_on='farmer_id', how='left')
sowing_agent_farmer.fillna(0)

phase1_df = sowing_agent_farmer[sowing_agent_farmer['cultivation_phase_id']==1].drop(columns=['cultivation_phase_id'])
phase1_df = phase1_df.rename(columns={'area':'total_nursury'})
phase1_df = phase1_df.groupby(['agent_id','agent_name']).agg({'total_nursury':sum, }).reset_index()

phase2_df = sowing_agent_farmer[sowing_agent_farmer['cultivation_phase_id']==2].drop(columns=['cultivation_phase_id'])
phase2_df = phase2_df.rename(columns={'area':'total_transplant'})
phase2_df = phase2_df.groupby(['agent_id']).agg({ 'sowing_id':'first','total_transplant':sum, 'geo_fence':sum}).reset_index()

sowing_phase_df = pd.merge(phase1_df, phase2_df, left_on='agent_id', right_on='agent_id', how='left')
sowing_phase_df['sowing_id'] = sowing_phase_df['sowing_id'].astype(int)

final_sowing_df = pd.merge(sowing_phase_df, final_df, left_on='agent_id', right_on='agent_id', how='left')
final_sowing_df1 = final_sowing_df.drop(columns=['agent_name_y'])

final_sowing_df1 = final_sowing_df1.fillna(0)
final_sowing_df1

In [51]:
# combo_issue_request_agentmap_df

In [52]:
# combo_issue_request_othershop_obj_df

In [53]:
# input_combo_df

In [54]:
# inupt_merge_df

In [55]:
# final_df

In [56]:
# agent_farmer_df

In [57]:
# sowing_farmers_df

In [59]:
# sowing_agent_farmer

In [60]:
# phase1_df

In [61]:
# phase2_df

In [62]:
# sowing_phase_df

In [63]:
# final_sowing_df

In [64]:
# final_sowing_df1

In [85]:
agent_invetry_seed_obj = AgentInventory.objects.filter(season_id=2, combo_issue_request__input_combo__inputpart__name__input_type_id=1).order_by('agent_id')

agent_invetry_seed_values = list(agent_invetry_seed_obj.values_list('combo_issue_request', 'agent_id','agent__userprofile__code', 'agent__first_name', 'agent__last_name', 'combo_issue_request__input_combo_id','combo_issue_request__input_combo__name', 'combo_issue_request__quantity_in_numbers','combo_issue_request__input_combo__price', 'combo_issue_request__comboissuerequestagentmap__shop__type__name'))
agent_invetry_columns = ['combo_issue_request', 'agent_id', 'agent_code', 'agent_first_name', 'agent_last_name', 'input_combo_id','product_name', 'product_qty','price_per_item', 'shop_name']
agent_invetry_seed_df = pd.DataFrame(agent_invetry_seed_values, columns=agent_invetry_columns)
agent_invetry_seed_df['seed_amount'] = (agent_invetry_seed_df['product_qty'] * agent_invetry_seed_df['price_per_item'])
agent_invetry_seed_df = agent_invetry_seed_df.groupby('agent_id').agg({'agent_code':'first','agent_first_name':'first', 'seed_amount':sum}).reset_index()

agent_invetry_seed_df

,agent_id,agent_code,agent_first_name,seed_amount


In [96]:
agent_invetry_obj = AgentInventory.objects.filter(season_id=2).order_by('agent_id')
input_combo_type_list = []
for i in agent_invetry_obj:
    data_dict = {
        "input_combo_id":'',
        "input_type_name": '',
        "input_type_id": ''
    }
    data_dict["input_combo_id"] = i.combo_issue_request.input_combo.id
    combo_name_obj = InputPart.objects.filter(input_combo_id=i.combo_issue_request.input_combo.id)
    if combo_name_obj.count() > 1:
        data_dict["input_type_name"] = 'Fertilizers Amount'
        data_dict["input_type_id"] = 3
    else:
        data_dict["input_type_name"] = combo_name_obj[0].name.input_type.name + " Amount"
        data_dict["input_type_id"] = combo_name_obj[0].name.input_type_id
    input_combo_type_list.append(data_dict)
input_combo_df = pd.DataFrame(input_combo_type_list)
input_combo_df = input_combo_df.drop_duplicates()
if input_combo_df.empty:
    input_combo_df = pd.DataFrame(columns=['input_combo_id', 'input_type_name', 'input_type_id'])
    
agent_inventry_list = agent_invetry_obj.values_list('agent_id','agent__userprofile__code', 'agent__first_name', 'agent__last_name', 'combo_issue_request__input_combo_id','combo_issue_request__input_combo__name', 'combo_issue_request__quantity_in_numbers','combo_issue_request__input_combo__price', 'combo_issue_request__comboissuerequestagentmap__shop__type__name')
agent_invetry_columns = ['agent_id', 'agent_code', 'agent_first_name', 'agent_last_name', 'input_combo_id','product_name', 'product_qty','price_per_item', 'shop_name']
agent_invetry_df = pd.DataFrame(agent_inventry_list, columns=agent_invetry_columns)
agent_invetry_df['sub_total'] = agent_invetry_df['product_qty'] * agent_invetry_df['price_per_item'] 

final_df = pd.merge(agent_invetry_df, input_combo_df, left_on='input_combo_id', right_on='input_combo_id', how='left')
final_df = final_df.groupby('input_type_id').agg({'agent_id':'first', 'agent_code':'first', 'agent_first_name':'first','input_type_name': 'first', 'sub_total': sum}).reset_index()
final_df = final_df.drop(columns=['input_type_id'])

#convert_groupby_table_product_cost_row_wise_value_into_column(using_pandas_pivot_table)
final_df = pd.pivot_table(final_df, index=['agent_id', 'agent_code', 'agent_first_name'] , columns='input_type_name', aggfunc=min, fill_value=0)

#convert_pivot_table_to_normal_df
final_df.columns = final_df.columns.droplevel(0) #remove amount
final_df.columns.name = None  #remove categories
final_df = final_df.reset_index() #index to columns

if not 'Seeds Amount' in final_df.columns:
    final_df['Seeds Amount'] = 0
if not 'Fertilizers Amount' in final_df.columns:
    final_df['Fertilizers Amount'] = 0
if not 'Agrochemicals Amount' in final_df.columns:
    final_df['Agrochemicals Amount'] = 0
    
final_df.rename(columns = {'Agrochemicals Amount':'Pesticides Amount'}, inplace = True)
final_df

,agent_id,agent_code,agent_first_name,Seeds Amount,Fertilizers Amount,Pesticides Amount


In [86]:
agent_invetry_pesticides_obj = AgentInventory.objects.filter(season_id=2, combo_issue_request__input_combo__inputpart__name__input_type_id=3).order_by('agent_id')
agent_invetry_pesticides_values = list(agent_invetry_pesticides_obj.values_list('agent_id', 'combo_issue_request', 'combo_issue_request__input_combo_id', 'combo_issue_request__input_combo__name', 'combo_issue_request__quantity_in_numbers','combo_issue_request__input_combo__price', 'combo_issue_request__comboissuerequestagentmap__shop__type__name'))
agent_invetry_pesticides_columns = ['agent_id','combo_issue_request', 'input_combo_id', 'product_name', 'product_qty','price_per_item', 'shop_name']
agent_invetry_pesticides_df = pd.DataFrame(agent_invetry_pesticides_values, columns=agent_invetry_pesticides_columns)
agent_invetry_pesticides_df = agent_invetry_pesticides_df.drop_duplicates(subset=['combo_issue_request'])
agent_invetry_pesticides_df['pesticides_amount'] = (agent_invetry_pesticides_df['product_qty'] * agent_invetry_pesticides_df['price_per_item'])
agent_invetry_pesticides_df = agent_invetry_pesticides_df.groupby('agent_id').agg({'pesticides_amount':sum}).reset_index()
agent_invetry_pesticides_df


,agent_id,pesticides_amount


In [87]:
agent_invetry_fertilizer_obj = AgentInventory.objects.filter(season_id=2, combo_issue_request__input_combo__inputpart__name__input_type_id=3).order_by('agent_id', )
agent_invetry_fertilizer_values = list(agent_invetry_fertilizer_obj.values_list('combo_issue_request', 'agent_id', 'combo_issue_request__input_combo_id', 'combo_issue_request__input_combo__name', 'combo_issue_request__quantity_in_numbers','combo_issue_request__input_combo__price', 'combo_issue_request__comboissuerequestagentmap__shop__type__name'))
agent_invetry_fertilizer_columns = ['combo_issue_request', 'agent_id', 'input_combo_id', 'product_name', 'product_qty','price_per_item', 'shop_name']
agent_invetry_fertilizer_df = pd.DataFrame(agent_invetry_fertilizer_values, columns=agent_invetry_fertilizer_columns)
agent_invetry_fertilizer_df = agent_invetry_fertilizer_df.drop_duplicates(subset=['combo_issue_request'])
agent_invetry_fertilizer_df['fertilizer_amount'] = (agent_invetry_fertilizer_df['product_qty'] * agent_invetry_fertilizer_df['price_per_item'])
agent_invetry_fertilizer_df = agent_invetry_fertilizer_df.groupby('agent_id').agg({'fertilizer_amount':sum}).reset_index()
agent_invetry_fertilizer_df

,agent_id,fertilizer_amount


In [88]:
procurement_group_obj = ProcurementGroup.objects.filter(season_id=2)
procurement_group_values = list(procurement_group_obj.values_list('agent_id', 'cost'))
procurement_group_columns = ["agent_id", "total_flower_amount"]
procurement_group_df = pd.DataFrame(procurement_group_values, columns=procurement_group_columns)
procurement_group_df = procurement_group_df.groupby('agent_id').agg({'total_flower_amount':sum}).reset_index()
procurement_group_df

,agent_id,total_flower_amount
0,31,1125362.40
1,32,3718319.10
2,34,1779297.90
3,35,19367444.10
4,36,14911.80
5,37,3269616.60
6,38,343223.70
7,40,4637961.30
8,41,3291316.50
9,43,5795868.00


In [4]:
Cluster.objects.all().values()

<QuerySet [{'id': 2, 'name': 'Terakanambi', 'notes': 'Gundlupet Zone'}, {'id': 5, 'name': 'Yadalaya', 'notes': 'Sargur Zone'}, {'id': 7, 'name': 'HD Kote', 'notes': 'Sargur Zone'}, {'id': 8, 'name': 'Nanjangud', 'notes': 'Sargur Zone'}, {'id': 4, 'name': 'Baragi', 'notes': 'Gundlupet Zone'}, {'id': 6, 'name': 'Sargur', 'notes': 'Sargur Zone'}, {'id': 9, 'name': 'Talavadi', 'notes': 'talavadi'}, {'id': 10, 'name': 'masinakudi', 'notes': 'masinakudi'}, {'id': 1, 'name': 'gopalpura', 'notes': 'Gundlupet Zone (beerambadi'}, {'id': 3, 'name': 'kodasoge', 'notes': 'Gundlupet Zone (bommanahali)'}, {'id': 11, 'name': 'Arakalavadi', 'notes': 'Arakalavadi'}]>

In [4]:
    agent_cluster_list = list(UserClusterMap.objects.filter(user__userprofile__user_type_id=6, season_id=4).values_list('user_id', 'user__first_name', 'cluster__name'))
    agent_cluster_df = pd.DataFrame(agent_cluster_list, columns=['Agent Id', 'Agent Name', 'Cluster Name']).sort_values('Cluster Name')

    agent_farmer_village_list = list(AgentFarmerMap.objects.filter(agent_id__in = agent_cluster_df['Agent Id'], farmer__season_id=3).values_list('agent_id', 'farmer__farmer__village__name'))
    agent_farmer_village_df = pd.DataFrame(agent_farmer_village_list, columns=['Agent Id', 'Village'])
    agent_farmer_village_df = agent_farmer_village_df.groupby('Agent Id').agg({'Village': 'count'}).reset_index()

    final_df = pd.merge(agent_cluster_df, agent_farmer_village_df, left_on='Agent Id', right_on='Agent Id', how='outer')
    final_df = final_df.fillna(0)
    final_df = final_df.drop(columns=['Agent Id'])
    final_dict = final_df.groupby('Cluster Name').apply(lambda x: x.to_dict('r')).to_dict()

    writer = pd.ExcelWriter(str("static/media/") + "cluster_wise_agent_village_count.xlsx", engine="xlsxwriter")

    for index, title in enumerate(final_dict):
        cluster_df = pd.DataFrame(final_dict[title])
        cluster_df.index += 1
        total = cluster_df.sum(numeric_only=True)
        total.name = 'Total'
        cluster_df = cluster_df.append(total.transpose())

        # creating excel sheet with name
        cluster_df.to_excel(writer, sheet_name=title, startrow=1)
        # assigning that sheet to obj
        workbook = writer.book
        worksheet = writer.sheets[title]
        merge_format = workbook.add_format(
            {
                "bold": 1,
                "border": 1,
                "align": "center",
                "valign": "vcenter",
                "fg_color": "yellow",
            }
        )

        date = datetime.datetime.now().date()
        # Merge 3 cells.
        worksheet.merge_range("A1:D1", "Cluster Wise Agent Village Count" + " - " + str(date), merge_format)

        format1 = workbook.add_format({"num_format": "#,##0.00"})

        # Set the column width and format.
        worksheet.set_column("B:B", 18, format1)
        worksheet.set_column(0, 17, 20)

        # Add a header format.
        header_format = workbook.add_format({"fg_color": "#D7E4BC"})

        # Write the column headers with the defined format.
        for col_num, value in enumerate(cluster_df.columns.values):
            worksheet.write(0, col_num + 1, value, header_format)
    writer.save()


Agent Id    55
Village     55
dtype: int64

In [49]:
input_type_list = list(InputType.objects.all().values_list('id', 'name'))
input_type_column = ['Input Type Id', 'Input Type Name']
input_type_df = pd.DataFrame(input_type_list, columns=input_type_column)

input_goods_obj = InputGoods.objects.all().order_by('id')
input_goods_list = list(
    input_goods_obj.values_list('id', 'code', 'input_name__input_type', 'input_name__name','quantity_now', 'quantity_at_receipt',
                                'number_of_units', 'date_of_expiry', 'date_of_manufacturing', 'date_of_receipt',
                                'supplier__name', 'cost', 'unit__name'))
input_goods_column = ['Id', 'Code', 'Input Type Id', 'Name Of Good', 'Total Qty', 'Received Qty', 'Number Of Units',
                      'Date Of Expiry', 'Date Of Manufacture', 'Date Of Receipt', 'Supplier Name', 'Cost', 'Unit Name']
input_goods_df = pd.DataFrame(input_goods_list, columns=input_goods_column)
# input_goods_df['Is Goods Taken'] = "False"
# input_goods_df.loc[input_goods_df['Total Qty'] != input_goods_df['Received Qty'], 'Is Goods Taken'] = "True"

# final_df = pd.merge(input_goods_df, input_type_df, left_on='Input Type Id', right_on='Input Type Id', how='left') 
# final_df = final_df.fillna(0)

# writer = pd.ExcelWriter(str("static/media/") + "inward_goods_ledger.xlsx", engine="xlsxwriter")

# for input_type in list(set(list(final_df['Input Type Name']))):
#     input_df = final_df[final_df['Input Type Name'] == input_type].reset_index(drop=True)
#     input_df = input_df.drop(columns=['Input Type Name', 'Unit Name', 'Is Goods Taken', 'Id', 'Input Type Id'])
#     input_df['Cost'] = input_df['Cost'].astype(float)
#     input_df['Number Of Units'] = input_df['Number Of Units'].astype(str)

#     input_df.index += 1
#     total = input_df.sum(numeric_only=True)
#     total.name = 'Total'
#     input_df = input_df.append(total.transpose())

#     # creating excel sheet with name
#     input_df.to_excel(writer, sheet_name=input_type, startrow=1)
#     # assigning that sheet to obj"
#     workbook = writer.book
#     worksheet = writer.sheets[input_type]
#     merge_format = workbook.add_format(
#         {
#             "bold": 1,
#             "border": 1,
#             "align": "center",
#             "valign": "vcenter",
#             "fg_color": "yellow",
#         }
#     )

#     date = datetime.datetime.now().date()
#     # Merge 3 cells.
#     worksheet.merge_range("A1:K1", "Inward Goods Ledger" + " - " + str(date), merge_format)

#     format1 = workbook.add_format({"num_format": "#,##0.00"})

#     # Set the column width and format.
#     worksheet.set_column("B:B", 18, format1)
#     worksheet.set_column(0, 17, 20)

#     # Add a header format.
#     header_format = workbook.add_format({"fg_color": "#D7E4BC"})

#     # Write the column headers with the defined format.
#     for col_num, value in enumerate(input_df.columns.values):
#         worksheet.write(0, col_num + 1, value, header_format)
# writer.save()

input_goods_df

,Id,Code,Input Type Id,Name Of Good,Total Qty,Received Qty,Number Of Units,Date Of Expiry,Date Of Manufacture,Date Of Receipt,Supplier Name,Cost,Unit Name
0,6,IG21_F0001,3,17:17:17,0.00,30000.00,600,2022-01-01,2021-01-01,2021-03-25,MALLIKARJUNASWAMY TRADERS,739200.00,kg
1,8,IG21_A0002,2,Tricoderma,0.12,80.00,1,2021-09-30,2021-03-30,2021-04-01,ICARKVK,None,kg
2,9,IG21_A0003,2,Psuedomanos,0.12,80.00,1,2021-09-30,2021-03-30,2021-04-01,ICARKVK,None,kg
3,10,IG21_A0004,2,PSB(Phosphate Solubilizing Bacteria),0.12,80.00,1,2021-09-30,2021-03-30,2021-04-01,ICARKVK,None,kg
4,17,IG21_A0009,2,Spreadmax(N-80),254.00,350.00,70,2023-03-30,2021-03-30,2020-04-08,KRUSHIAGRO TECHNOLOGIES,None,lit
...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,95,IG21_F0041,3,10:26:26,86000.00,86000.00,1720,2022-04-29,2021-04-30,2021-06-15,MALLIKARJUNASWAMY TRADERS,1995200.00,kg
82,96,IG22_S0009,1,Seed,299500.00,300000.00,300,2022-02-25,2022-02-11,2022-02-09,test,3000.00,kg
83,97,IG22_A0047,2,biozyme,298750.00,300000.00,300,2022-02-26,2022-02-19,2022-02-17,test,3000.00,kg
84,98,IG22_F0042,3,MKP,300000.00,300000.00,300,2022-02-28,2022-02-18,2022-02-14,test,None,kg


In [32]:
Code, Name, Quantity at hand, Received Quantity, Received Units, Expiry, Manufactured on, Received on, SupplierCost (Rs)

SyntaxError: invalid syntax (<ipython-input-32-efa5bde8f9cd>, line 1)

In [47]:
season_id = 3
agent_id = 140
net_weignt = "4"


agent_procurement_list = ProcurementGroup.objects.filter(season_id=3, agent_id=140).values_list('agent__first_name', 'procurement_date', 'procurement_produce__name',
                                                                                                'season__name', 'produce_net_weight', 'price_per_unit', 'cost', 'agent_price_deduction', 
                                                                                                'payment_to_wallet', 'payment_to_agent')
agent_procurement_column = ['Agent Name', 'Procurement Date', 'Procurement Produce Name','Season', 'Produce Net Weight', 'Price Per Unit', 'Cost', 'Agent Price Deduction', 
                                                                                                'Payment To Wallet', 'Payment To Agent']  
agent_procurement_df = pd.DataFrame(agent_procurement_list, columns=agent_procurement_column)

agent_procurement_df["Net Weight In Ton"] = agent_procurement_df["Produce Net Weight"]/1000

agent_procurement_df = agent_procurement_df.fillna(0)

if net_weight == "4":
     final_df = agent_procurement_df[(agent_procurement_df['Net Weight In Ton'] >= 4) & (agent_procurement_df['Net Weight In Ton']/1000 <=6)]
if net_weight == 6:
     final_df = agent_procurement_df[(agent_procurement_df['Net Weight In Ton'] > 6) & (agent_procurement_df['Net Weight In Ton']/1000 <=8)]
if net_weight == 8:
     final_df = agent_procurement_df[(agent_procurement_df['Net Weight In Ton'] > 8)]

writer = pd.ExcelWriter(str("static/media/") + "agent_wise_procurement_weight_report.xlsx", engine="xlsxwriter")

final_df = final_df.reset_index(drop=True)
final_df[['Produce Net Weight', 'Price Per Unit', 'Cost', 'Agent Price Deduction','Payment To Wallet', 'Payment To Agent', 'Net Weight In Ton']] = final_df[['Produce Net Weight', 'Price Per Unit', 'Cost', 'Agent Price Deduction','Payment To Wallet', 'Payment To Agent', 'Net Weight In Ton']].astype(float)

final_df.index += 1
total = final_df.sum(numeric_only=True)
total.name = 'Total'
final_df = final_df.append(total.transpose())

# creating excel sheet with name
final_df.to_excel(writer, sheet_name="sheet1", startrow=1)
# assigning that sheet to obj"
workbook = writer.book
worksheet = writer.sheets["sheet1"]
merge_format = workbook.add_format(
    {
        "bold": 1,
        "border": 1,
        "align": "center",
        "valign": "vcenter",
        "fg_color": "yellow",
    }
)

date = datetime.datetime.now().date()
# Merge 3 cells.
worksheet.merge_range("A1:L1", "Agent Wise Procurement Weight Report" + " - " + str(date), merge_format)

format1 = workbook.add_format({"num_format": "#,##0.00"})

# Set the column width and format.
worksheet.set_column("B:B", 18, format1)
worksheet.set_column(0, 17, 20)

# Add a header format.
header_format = workbook.add_format({"fg_color": "#D7E4BC"})

# Write the column headers with the defined format.
for col_num, value in enumerate(final_df.columns.values):
    worksheet.write(0, col_num + 1, value, header_format)
writer.save()

In [45]:
final_df.columns
final_df[['Produce Net Weight', 'Price Per Unit', 'Cost', 'Agent Price Deduction','Payment To Wallet', 'Payment To Agent', 'Net Weight In Ton']] = final_df[['Produce Net Weight', 'Price Per Unit', 'Cost', 'Agent Price Deduction','Payment To Wallet', 'Payment To Agent', 'Net Weight In Ton']].astype(float)

Index(['Agent Name', 'Procurement Date', 'Procurement Produce Name', 'Season',
       'Produce Net Weight', 'Price Per Unit', 'Cost', 'Agent Price Deduction',
       'Payment To Wallet', 'Payment To Agent', 'Net Weight In Ton'],
      dtype='object')

In [20]:
import re
user_type_id = 16

if PositionPositionUserMap.objects.filter(user__userprofile__user_type_id=user_type_id).exists():
    last_position_name = PositionPositionUserMap.objects.filter(user__userprofile__user_type_id=user_type_id).order_by('-id').values_list('position__name', flat=True)[0]
    split_numbers_and_strings = re.match(r"([a-z]+)([0-9]+)", last_position_name, re.I)
    if split_numbers_and_strings:
        items = list(split_numbers_and_strings.groups())
        if items[-1][0] == '0':
            current_position_name = items[0] + '0' + str(int(items[-1])+1)
        else:
            current_position_name = items[0] + str(int(items[-1])+1)
    else:
        current_position_name = last_position_name + '02'
    print(current_position_name)
else:
    user_type_position_obj = UserType.objects.get(id=user_type_id)
    position_obj = Position(name=user_type_position_obj.short_name + '01', notes=user_type_position_obj.name, is_active=True)
    position_obj.save()

SV01


In [4]:
import re
split_numbers_and_strings = re.match(r"([a-z]+)([0-9]+)", last_position_name, re.I)
if split_numbers_and_strings:
    items = list(split_numbers_and_strings.groups())


In [5]:
last_position_name = PositionPositionUserMap.objects.filter(user__userprofile__user_type_id=15).order_by('-id').values_list('position__name', flat=True)[0]
split_numbers_and_strings = re.match(r"([a-z]+)([0-9]+)", last_position_name, re.I)
if split_numbers_and_strings:
    items = list(split_numbers_and_strings.groups())
current_position_name = items[0] + str(int(items[-1])+1)
current_position_name

NameError: name 'items' is not defined

In [6]:
list(UserType.objects.all().values('id', 'name', 'short_name'))

[{'id': 3, 'name': 'Senior Field Supervisor', 'short_name': '0'},
 {'id': 6, 'name': 'Agent', 'short_name': '0'},
 {'id': 7, 'name': 'Farmer', 'short_name': '0'},
 {'id': 8, 'name': 'Software Developers', 'short_name': '0'},
 {'id': 4, 'name': 'Agriculture Officer', 'short_name': '0'},
 {'id': 5, 'name': 'Asst Field Supervisor', 'short_name': '0'},
 {'id': 10, 'name': 'Partner', 'short_name': '0'},
 {'id': 12, 'name': 'Supply chain executive', 'short_name': '0'},
 {'id': 14, 'name': 'Accounts Executive', 'short_name': '0'},
 {'id': 13, 'name': 'Accounts Officer', 'short_name': '0'},
 {'id': 15, 'name': 'Weigh Bridge Operator', 'short_name': '0'},
 {'id': 1, 'name': 'General Manager', 'short_name': 'GM'},
 {'id': 2, 'name': 'Assistant Manager (Agri)', 'short_name': 'AMA'},
 {'id': 16, 'name': 'Supervisor', 'short_name': 'SV'}]

In [10]:
short_name_list = [{'id': 3, 'name': 'Senior Field Supervisor', 'short_name': 'SFS'},
 {'id': 6, 'name': 'Agent', 'short_name': 'AG'},
 {'id': 7, 'name': 'Farmer', 'short_name': 'FM'},
 {'id': 8, 'name': 'Software Developers', 'short_name': 'SD'},
 {'id': 4, 'name': 'Agriculture Officer', 'short_name': 'AO'},
 {'id': 5, 'name': 'Asst Field Supervisor', 'short_name': 'AFS'},
 {'id': 10, 'name': 'Partner', 'short_name': 'PT'},
 {'id': 12, 'name': 'Supply chain executive', 'short_name': 'SCE'},
 {'id': 14, 'name': 'Accounts Executive', 'short_name': 'AE'},
 {'id': 13, 'name': 'Accounts Officer', 'short_name': 'ACO'},
 {'id': 15, 'name': 'Weigh Bridge Operator', 'short_name': 'WBO'},
 {'id': 1, 'name': 'General Manager', 'short_name': 'GM'},
 {'id': 2, 'name': 'Assistant Manager (Agri)', 'short_name': 'AMA'},
 {'id': 16, 'name': 'Supervisor', 'short_name': 'SV'}]

for i in short_name_list:
    user_type_obj = UserType.objects.get(id=i['id'])
    user_type_obj.short_name = i['short_name']
    user_type_obj.save()

In [46]:
hobli_village_df = pd.DataFrame(list(Hobli.objects.all().values_list('name', 'village__name')), columns=['Hobli', 'Village']) 
hobli_village_dict = hobli_village_df.groupby('Hobli').apply(lambda x:x.to_dict('r')).to_dict()

writer = pd.ExcelWriter(str("static/media/") + "hobli_wise_village.xlsx", engine="xlsxwriter")

for hobli in hobli_village_dict:
    print(hobli)
    final_df = pd.DataFrame(hobli_village_dict[hobli])
    final_df= final_df.sort_values('Village')
    
#     final_df.index += 1

    # creating excel sheet with name
    final_df.to_excel(writer, sheet_name=hobli, startrow=1, index=None)
    # assigning that sheet to obj"
    workbook = writer.book
    worksheet = writer.sheets[hobli]
    merge_format = workbook.add_format(
        {
            "bold": 1,
            "border": 1,
            "align": "center",
            "valign": "vcenter",
            "fg_color": "yellow",
        }
    )

    date = datetime.datetime.now().date()
    # Merge 3 cells.
    worksheet.merge_range("A1:B1", "Hobli Wise Village Report" + " - " + str(date), merge_format)

    format1 = workbook.add_format({"num_format": "#,##0.00"})

    # Set the column width and format.
    worksheet.set_column("B:B", 18, format1)
    worksheet.set_column(0, 17, 20)

    # Add a header format.
    header_format = workbook.add_format({"fg_color": "#D7E4BC"})

    # Write the column headers with the defined format.
    for col_num, value in enumerate(final_df.columns.values):
        worksheet.write(0, col_num + 1, value, header_format)
writer.save()



Alathur
Antarasante
Antharasanthe
Begur
Beguru
Berambadi
Bisalavadi
Chandakavadi
Chikkahalli
Doddakavalande
Gddige
Gobichettipalayam
Guddekeri
Gundlupet
Gundlupet town
H d kote
Hampapura
Handpost
Hangala
Hangla
Hanur
Haradanahalli
Haradhanahalli
Harave
Hardhanahalli
Harve
Hullahalli
Hura
Igglore
Jayapura
Kamakerai
Kandalike
Kannegala
Kasaba
Kavalande
Kavalandi
Kavlande
Kodasoge
Kollegal
Kowlande
Kunagahalli
Kurumbampalayam
Kuvempunagar
Kuvempunagara
Lokkanahalli
Masanigudi
Masinagudi
Muguru
N begur
Santhemarahalli
Sargur
Satyamangalam
Savukanahalli
T narasipura
Tagadur
Talamalai
Talavadi
Terakanambi
Thoravalli
Tirukanambi
Yalanduru
Yarahalli handpost


<ipython-input-46-47629056a6e4>:2: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  hobli_village_dict = hobli_village_df.groupby('Hobli').apply(lambda x:x.to_dict('r')).to_dict()


In [17]:
season_id = get_active_season_id()
master_dict = {}

active_sfs_ids = list(UserHierarchyMap.objects.filter(superior_user_type_id=3, season_id=season_id).values_list('superior_id', flat=True))

active_users = list(UserProfile.objects.filter(user_id__in=active_sfs_ids).values_list('user_id', 'user__username', 'user__last_name'))
active_users_columns = ['user_id', 'first_name', 'last_name']
active_users_df = pd.DataFrame(active_users, columns=active_users_columns)
master_dict['active'] = active_users_df.to_dict('r')

in_active_sfs_ids = list(UserHierarchyMap.objects.filter(superior_user_type_id=3).values_list('superior_id', flat=True))
in_active_users = list(UserProfile.objects.filter(user_id__in=in_active_sfs_ids).exclude(user_id__in=active_sfs_ids).values_list('user_id', 'user__username', 'user__last_name'))
in_active_users_columns = ['user_id', 'first_name', 'last_name']
in_active_users_df = pd.DataFrame(in_active_users, columns=in_active_users_columns)
in_active_users_df = in_active_users_df.drop_duplicates(subset=['user_id'])
master_dict['in_active'] = in_active_users_df.to_dict('r')



<ipython-input-17-1458be949960>:9: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  master_dict['active'] = active_users_df.to_dict('r')


,user_id,first_name,last_name
0,92,mahadevaprasad_sfs2,sfs2


In [9]:
season_obj = Season.objects.get(is_active=True)
season_obj.id

4

In [3]:
def get_active_season_id():
    season_obj = Season.objects.get(is_active=True)
    return season_obj.id

In [4]:
def generate_agent_code(user_type_id, season_id, cluster_id):
    print(UserProfileCodeBank.objects.get(season_id=season_id,user_type_id=user_type_id).last_count)
    print(UserProfileCodeBank.objects.get(season_id=season_id,user_type_id=user_type_id).prefix_code)
    print(Cluster.objects.get(id=cluster_id).name[:1])
    prefix = UserProfileCodeBank.objects.get(season_id=season_id,user_type_id=user_type_id).prefix_code
    if user_type_id == 6:
        prefix = prefix + str(Cluster.objects.get(id=cluster_id).name[:1])
    print('prefix: ', prefix)
    last_count =UserProfileCodeBank.objects.get(season_id=season_id,user_type_id=user_type_id).last_count
    generated_count = last_count + 1
    suffix = str(generated_count).zfill(3)
    code = str(prefix) + suffix
    UserProfileCodeBank.objects.filter(season_id=season_id,user_type_id=user_type_id).update(last_count=generated_count)
    print(code)
    return code

In [5]:
sheet_id = '1jt1owSgaONho2R7BPIpjehVPtMwZ2xe3OLx1TSXeEfQ'
sheet_name = "Sheet1"
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
agent_df = pd.read_csv(url).fillna('')
agent_df

season_id = get_active_season_id()

for index, value in agent_df.iterrows():    
    if User.objects.filter(username=value["mobile"]).exists() or value['mobile'] == 0:
        print("This Agent already Register or Don't have phone number ", value['first_name'], index)
    else:
        # create user
        user_obj = User(
            first_name=value["first_name"],
            last_name=value["last_name"],
            username=value["mobile"],
            password=make_password(1234),
        )
        user_obj.save()
        print("user created success")
        
        # create user profile
        season_id = get_active_season_id()
        user_profile = UserProfile(
            business_id=1,
            user=user_obj,
            code=generate_agent_code(6,season_id, 1),
            user_type_id=6,
            mobile=value["mobile"],
            state_id=1,
            district_id=1,
            taluk_id=1,
            address=value["address"],
            pincode=641107,
            hobli_id=1,
            aadhaar_number=value["aadhaar_number"],
            created_by_id=1,
            modified_by_id=1,
        )

        user_profile.save()
        print("user profile saved")
        
        UserBankDetails.objects.create(
            user=user_obj,
            bank=value['bank'],
            branch=value['branch'],
            ifsc_code=value['ifsc_code'],
            account_holder_name=value['account_holder_name'],
            account_number=value['account_number'],
            created_by_id=1,
            modified_by_id=1
        )


        AgentMergeloadEnable.objects.create(season_id=season_id, agent_id=user_obj.id)
        
        #agentwallet
        AgentWallet.objects.create(modified_by_id=request.user.id, agent_id=user_obj.id)
        print("agent save success")


This Agent already Register or Don't have phone number  Basavanna 0
user created success
1
A
G
prefix:  AG
AG002
user profile saved
agent save success
user created success
2
A
G
prefix:  AG
AG003
user profile saved
agent save success
user created success
3
A
G
prefix:  AG
AG004
user profile saved
agent save success
user created success
4
A
G
prefix:  AG
AG005
user profile saved
agent save success
user created success
5
A
G
prefix:  AG
AG006
user profile saved
agent save success
user created success
6
A
G
prefix:  AG
AG007
user profile saved
agent save success
user created success
7
A
G
prefix:  AG
AG008
user profile saved
agent save success
user created success
8
A
G
prefix:  AG
AG009
user profile saved
agent save success
This Agent already Register or Don't have phone number  Ravi Kumara 9
This Agent already Register or Don't have phone number  Ramesha 10
This Agent already Register or Don't have phone number  Madhukumar 11
user created success
9
A
G
prefix:  AG
AG010
user profile sav

In [6]:
agent = UserClusterMap.objects.filter(season_id=4, cluster_id=1, user__userprofile__user_type_id=6)
agent


<QuerySet []>